First Phase Plan:
- Figure out how we want to use the data (create our own dataset? use a large one from kaggle for new songs?)
- Start with a smaller dataset containing x songs from each person (eg: 10 songs from a playlist that belongs to Coralie, 10 from Tamara and 10 from Prof Boily)
- Create a simple model that, when a given a song, it will "classify" it as the playlist it should belong to
- Explore which features or which combination of features allow for high quality classification (eg: artist, genre, year, lyrics, or a combination of them)

Future phases ideas:
- Possibility to use sentiment analysis for lyrics??
- Build this into something more?

# Book-Crossings Dataset
The Book-Crossing dataset is a well-known dataset in the recommendation systems and data science community. It was collected by Cai-Nicolas Ziegler in 2004 and consists of three main files:

1. Users (BX-Users.csv): Contains demographic information such as User-ID, Location, and Age.
2. Books (BX-Books.csv): Contains details about books, including ISBN, Book-Title, Book-Author, Year-Of-Publication, Publisher, and a URL for the book cover.
3. Ratings (BX-Book-Ratings.csv): Contains explicit ratings given by users, with User-ID, ISBN, and Book-Rating (ranging from 0 to 10, where 0 means implicit interaction).

https://www.kaggle.com/datasets/syedjaferk/book-crossing-dataset?select=BX-Users.csv

# TOY PROBLEM
To start, we are going to design a small example problem acting as a course recommender system. We want to build something that has a similar structure to our Book-Crossings dataset so that we have a way to use it for that task eventually.

## First Step: Building a Toy Dataset
Since we are building this dataset ourselves and that we are essentially using it as a basis for our future project, we want it to have a similar stucture to the book-crossings dataset:

1. Students Table (Users Equivalent): Student-ID, Year (Instead of Age), Major (Instead of location)
2. Courses Table (Books Equivalent): Course-ID (Instead of ISBN), Course-Title (Instead of Book-Title), Language (Adding an additional feature Instead of Author for example), Department (Instead of Publisher)
3. Ratings Table (Interactions): Student-ID, Course-ID, Rating (0-5 Scale, consistent with Rate my Prof scale that students are already familiar with).

Random values were used to generate the tables (with a subset of courses). As of this time, I have not checked to make sure that if their year is 1, they can't take/rate 4th year courses.



In [ ]:
import pandas as pd

# Load datasets from the sample_data folder
students_df = pd.read_csv('/content/sample_data/Students_Table.csv')
courses_df = pd.read_csv('/content/sample_data/Courses_Table.csv')
ratings_df = pd.read_csv('/content/sample_data/Ratings_Table.csv')

# Display the first few rows to verify the data
print("Students Table:")
display(students_df.head())

print("Courses Table:")
display(courses_df.head())

print("Ratings Table:")
display(ratings_df.head())


FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/Students_Table.csv'

In [ ]:
print("Students DataFrame Shape:", students_df.shape)
print("Courses DataFrame Shape:", courses_df.shape)
print("Ratings DataFrame Shape:", ratings_df.shape)

In [ ]:
print("Missing Values in Students:\n", students_df.isnull().sum())
print("Missing Values in Courses:\n", courses_df.isnull().sum())
print("Missing Values in Ratings:\n", ratings_df.isnull().sum())

In [ ]:
print("Unique Student IDs:", students_df['Student-ID'].nunique())
print("Unique Course IDs:", courses_df['Course-ID'].nunique())
print("Unique Ratings:", ratings_df['Rating'].unique())

# Book Crossing
We would like to create a hybrid recommender system, made up of content-based and collaborative filtering. First we start with content-based filtering, and later we will combine it with collaborative filtering. First we start with some data cleaning.

## Data Cleaning

In [ ]:
import pandas as pd

items_info_df = pd.read_csv('https://raw.githubusercontent.com/tamaraamicic/Honours-Project-Datasets/refs/heads/main/items_info.dat', sep='\t', on_bad_lines='skip', index_col=False)
items_info_df.head()

,Book_ID,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,1,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
1,3,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,http://images.amazon.com/images/P/0425176428.0...,http://images.amazon.com/images/P/0425176428.0...
2,4,0452264464,Beloved (Plume Contemporary Fiction),Toni Morrison,1994,Plume,http://images.amazon.com/images/P/0452264464.0...,http://images.amazon.com/images/P/0452264464.0...,http://images.amazon.com/images/P/0452264464.0...
3,5,0609804618,Our Dumb Century: The Onion Presents 100 Years...,The Onion,1999,Three Rivers Press,http://images.amazon.com/images/P/0609804618.0...,http://images.amazon.com/images/P/0609804618.0...,http://images.amazon.com/images/P/0609804618.0...
4,7,1879384493,If I'd Known Then What I Know Now: Why Not Lea...,J. R. Parrish,2003,Cypress House,http://images.amazon.com/images/P/1879384493.0...,http://images.amazon.com/images/P/1879384493.0...,http://images.amazon.com/images/P/1879384493.0...


In [ ]:
len(items_info_df)

16411

The original file has 17384 rows, but the length of items_info_df is 16411, since we skip bad lines (on_bad_lines='skip'). We can inspect these rows by printing the ones that do not have the expected 9 columns:

In [ ]:
with open('/content/items_info.dat', 'r', encoding='utf-8') as file:
    for i, line in enumerate(file):
        col_count = len(line.strip().split("\t"))
        if col_count != 9: # 9 is the expected number of columns. print the rows where there are more then 9, so that we can inspect what is wrong with them
            print(f"Line {i+1} has {col_count} columns: {line.strip()}")

FileNotFoundError: [Errno 2] No such file or directory: '/content/items_info.dat'

We noticed that a lot of them have '&amp  ' (ampersand, 'amp', and a tab). The extra tab creates an extra column, since this is a tab separated file. Therefore we replace the '&amp ' (ampersand, 'amp', and a tab) with '& ' (ampersand and space):

In [ ]:
import re

input_file_path = "/content/items_info.dat"
output_file_path = "/content/items_info_cleaned.dat"

corrected_lines = []

with open(input_file_path, 'r', encoding='utf-8') as file:
    for i, line in enumerate(file):
        original_line = line.strip()  # remove trailing newline and spaces
        col_count = len(original_line.split("\t"))  # count number of columns
        corrected_line = re.sub(r"&amp[\t ]+", "& ", original_line) # matches one or more (+) tabs (\t) or spaces (" ") that follow "&amp", replace with "& "
        corrected_lines.append(corrected_line) # store the corrected line

# writing the cleaned data to a new file
with open(output_file_path, 'w', encoding='utf-8') as cleaned_file:
    for line in corrected_lines:
        cleaned_file.write(line + "\n")

print(f"Cleaned dataset saved to: {output_file_path}")

FileNotFoundError: [Errno 2] No such file or directory: '/content/items_info.dat'

Now lets look at this cleaned dataset:

In [ ]:
items_info_cleaned_df = pd.read_csv('/content/items_info_cleaned.dat', sep='\t', on_bad_lines='skip', index_col=False)
items_info_cleaned_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/items_info_cleaned.dat'

In [ ]:
len(items_info_cleaned_df)

NameError: name 'items_info_cleaned_df' is not defined

This new dataset is still missing 27 rows. Its length is 17357, whereas the original file had a length of 17384. So we inspect the remaining rows that do not have the expected 9 columns:

In [ ]:
with open('/content/items_info_cleaned.dat', 'r', encoding='utf-8') as file:
    for i, line in enumerate(file):
        col_count = len(line.strip().split("\t"))
        if col_count != 9: # 9 is the expected number of columns. print the rows where there are more then 9, so that we can inspect what is wrong with them
            print(f"Line {i+1} has {col_count} columns: {line.strip()}")

FileNotFoundError: [Errno 2] No such file or directory: '/content/items_info_cleaned.dat'

We notice extra tabs in random places, so we manually removed them from the items_info_cleaned.dat file. We stored this in a new file called items_info_cleaned_2.dat. Now when we inspect the rows that do not have the usual 9 columns, there aren't any! :

In [ ]:
with open('/content/items_info_cleaned_2.dat', 'r', encoding='utf-8') as file:
    for i, line in enumerate(file):
        col_count = len(line.strip().split("\t"))
        if col_count != 9: # 9 is the expected number of columns. print the rows where there are more then 9, so that we can inspect what is wrong with them
            print(f"Line {i+1} has {col_count} columns: {line.strip()}")

FileNotFoundError: [Errno 2] No such file or directory: '/content/items_info_cleaned_2.dat'

In [ ]:
import pandas as pd

items_info_cleaned_2_df = pd.read_csv('https://raw.githubusercontent.com/tamaraamicic/Honours-Project-Datasets/refs/heads/main/items_info_cleaned_2.dat', sep='\t', on_bad_lines='skip', index_col=False,)
items_info_cleaned_2_df.head()

,Book_ID,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,1,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
1,2,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
2,3,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,http://images.amazon.com/images/P/0425176428.0...,http://images.amazon.com/images/P/0425176428.0...
3,4,0452264464,Beloved (Plume Contemporary Fiction),Toni Morrison,1994,Plume,http://images.amazon.com/images/P/0452264464.0...,http://images.amazon.com/images/P/0452264464.0...,http://images.amazon.com/images/P/0452264464.0...
4,5,0609804618,Our Dumb Century: The Onion Presents 100 Years...,The Onion,1999,Three Rivers Press,http://images.amazon.com/images/P/0609804618.0...,http://images.amazon.com/images/P/0609804618.0...,http://images.amazon.com/images/P/0609804618.0...


The length is also the same as the original file:

In [ ]:
len(items_info_cleaned_2_df)

17384

Now we can start with content-based filtering.

# Content-Based Filtering

Content-based filtering recommends items based on their features. In this case, the books are the items, and features we can use are title, author, year of publication, and publisher to determine similarities between books. We can omit the rest for now (ISBN and Image-URL).

In [39]:
items_info_cleaned_2_df["Features"] = (
    items_info_cleaned_2_df["Book-Title"] + " " +
    items_info_cleaned_2_df["Book-Author"] + " " +
    items_info_cleaned_2_df["Publisher"]
)
items_info_cleaned_2_df[["Book-Title", "Features"]].head()

,Book-Title,Features
0,Decision in Normandy,Decision in Normandy Carlo D'Este HarperPerennial
1,The Mummies of Urumchi,The Mummies of Urumchi E. J. W. Barber W. W. N...
2,What If?: The World's Foremost Military Histor...,What If?: The World's Foremost Military Histor...
3,Beloved (Plume Contemporary Fiction),Beloved (Plume Contemporary Fiction) Toni Morr...
4,Our Dumb Century: The Onion Presents 100 Years...,Our Dumb Century: The Onion Presents 100 Years...


In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words="english") # initializing tf-idf vectorizer. stop_words="english" removes common english stopwords
tfidf_matrix = tfidf_vectorizer.fit_transform(items_info_cleaned_2_df["Features"]) # applies TF-IDF (term frequency/inverse document frequency) to convert the text into a matrix of numbers. each cell is the TF-IDF score (how important that word is for that book)

# rows = books, columns = words
print(tfidf_matrix.shape)

(17384, 22527)


In [41]:
# just to see what it looks like
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
tfidf_df.head(10)

,000,003,007,01,02,03,04,05,06,07959,...,â¼skind,â¼sse,â¼sselkindes,â¼sselworte,â¼ter,â¼tersloher,â¼ttern,ãµes,ãºltima,ã¼ber
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tfidf_matrix)

print(cosine_sim.shape)

(17384, 17384)


In [43]:
# just to see what it looks like
cosine_sim_df = pd.DataFrame(cosine_sim, index=items_info_cleaned_2_df.index, columns=items_info_cleaned_2_df.index)
cosine_sim_df.iloc[:10, :10]

,0,1,2,3,4,5,6,7,8,9
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.000000,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.560694,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.560694,1.000000,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1.0


In [44]:
book_indices = pd.Series(items_info_cleaned_2_df.index, index=items_info_cleaned_2_df["Book-Title"]).drop_duplicates() # mapping book titles to their corresponding index in the dataframe
book_indices

,0
Book-Title,
Decision in Normandy,0
The Mummies of Urumchi,1
What If?: The World's Foremost Military Historians Imagine What Might Have Been,2
Beloved (Plume Contemporary Fiction),3
Our Dumb Century: The Onion Presents 100 Years of Headlines from America's Finest News Source,4
...,...
Guide to Confident Living,17379
Ludwig Feuerbach,17380
To Bring You Joy (Ulverscroft Large Print Series),17381


In [45]:
def simple_content_based_filtering(title, num_recommendations):
    matches = items_info_cleaned_2_df[items_info_cleaned_2_df['Book-Title'] == title]

    if matches.empty:
        raise ValueError(f"'{title}' not found in the dataset.")

    if len(matches) > 1:
        print(f"Multiple entries found for '{title}'. Using the first one:")
        display(matches[['Book-Title', 'Book-Author', 'Publisher']].iloc[0])

    index = matches.index[0] # gets index of first match

    #index = book_indices[title] # get the given book
    sim_scores = list(enumerate(cosine_sim[index])) # similarity score between the given book and all the others
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:num_recommendations+1] # get most similar. exclude itself
    book_indices_list = [i[0] for i in sim_scores] # get indices of recommended books

    return items_info_cleaned_2_df.iloc[book_indices_list][["Book-Title", "Book-Author", "Publisher"]] # get relevant book details for display

simple_content_based_filtering("Decision in Normandy", 5)

,Book-Title,Book-Author,Publisher
12986,Decision at Doona,Anne McCaffrey,Del Rey Books
17061,The Eleventh Summer,Carlo Gebler,Simon & Schuster
2756,Due di due (Bestsellers),Andrea De Carlo,A. Mondadori
12328,Io Francesco,Carlo Carretto,Distribooks Inc
10374,Your Sacred Self: Making the Decision to Be Free,Wayne W. Dyer,HarperTorch


Ideas to refine this

- don't combine the features, and then we can do a weighted average later. so for example, maybe title similarity is more important than publisher similarity.
- do year similarity
- is title of book enough to identify a book? some books might have the same title. instead of drop duplicates maybe have a better identifier, such as title and author, or use the isbn or something like that
- question -- how are we going to test accuracy?

## Experimenting with friends' books

In [46]:
friend7_books = [
    "Harry Potter and the Philosopher's Stone",
    "Harry Potter and the Chamber of Secrets",
    "Harry Potter and the Prisoner of Azkaban",
    "Harry Potter and the Goblet of Fire",
    "Harry Potter and the Order of the Phoenix",
    "Harry Potter and the Half-Blood Prince",
    "Harry Potter and the Deathly Hallows",
    "Crazy Rich Asians",
    "Dance of Thieves",
    "Vow of Thieves",
    "Divergent",
    "City of Sparks",
    "The City of Ember",
    "Verity"
]

friend8_books = [
    "1984",
    "The Lord of the Rings",
    "The Five People You Meet in Heaven",
    "Verity",
    "Throne of Glass",
    "The Hunger Games",
    "Fourth Wing",  # "The Empyrean"??
    "Camp Zero",
    "The Midnight Library"
    "Life of Pi"
]

friend1_books = [
    "Handmaid's Tale",
    "Shutter Island",
    "Rum Punch",
    "L.A. Confidential",
    "Dracula",
    "Crime and Punishment",
    "Misery",
    "Catcher in The Rye",
    "The Godfather",
    "Frankenstein",
    "Shake Hands With The Devil",
    "The Great Gatsby",
    "Lord of the Flies",
    "War Of The Worlds"
]

In [47]:
def recommend_similar_books_with_simple_content(book_list, num_recs_per_book=3):
    recommendations = pd.DataFrame()
    for book in book_list:
        if book in book_indices:
            print("Book: ", book, "\n")
            recs = simple_content_based_filtering(book, num_recs_per_book)
            recs['Reference Book'] = book
            recommendations = pd.concat([recommendations, recs])
            print('recs: ', recommendations)
        else:
            print(f"'{book}' not found in dataset.")
    return recommendations.drop_duplicates(subset="Book-Title").reset_index(drop=True)

In [48]:
friend7_recs = recommend_similar_books_with_simple_content(friend7_books, num_recs_per_book=2)
friend8_recs = recommend_similar_books_with_simple_content(friend8_books, num_recs_per_book=2)
friend1_recs = recommend_similar_books_with_simple_content(friend1_books, num_recs_per_book=2)

'Harry Potter and the Philosopher's Stone' not found in dataset.
'Harry Potter and the Chamber of Secrets' not found in dataset.
'Harry Potter and the Prisoner of Azkaban' not found in dataset.
'Harry Potter and the Goblet of Fire' not found in dataset.
'Harry Potter and the Order of the Phoenix' not found in dataset.
'Harry Potter and the Half-Blood Prince' not found in dataset.
'Harry Potter and the Deathly Hallows' not found in dataset.
'Crazy Rich Asians' not found in dataset.
'Dance of Thieves' not found in dataset.
'Vow of Thieves' not found in dataset.
'Divergent' not found in dataset.
'City of Sparks' not found in dataset.
'The City of Ember' not found in dataset.
'Verity' not found in dataset.
Book:  1984 

recs:                                Book-Title    Book-Author  \
2761                         Animal Farm  George Orwell   
3977  1984 (Signet Classics (Paperback))  George Orwell   

                      Publisher Reference Book  
2761  New Amer Library Classics         

,11456
Book-Title,The Lord of the Rings
Book-Author,J. R. R. Tolkien
Publisher,Houghton Mifflin Company


recs:                                             Book-Title       Book-Author  \
2761                                      Animal Farm     George Orwell   
3977               1984 (Signet Classics (Paperback))     George Orwell   
17294                           The Lord of the Rings  J. R. R. Tolkien   
5401   The Two Towers (The Lord of the Rings, Part 2)  J. R. R. Tolkien   

                       Publisher         Reference Book  
2761   New Amer Library Classics                   1984  
3977             Sagebrush Bound                   1984  
17294   Houghton Mifflin Company  The Lord of the Rings  
5401    Houghton Mifflin Company  The Lord of the Rings  
'The Five People You Meet in Heaven' not found in dataset.
'Verity' not found in dataset.
'Throne of Glass' not found in dataset.
'The Hunger Games' not found in dataset.
'Fourth Wing' not found in dataset.
'Camp Zero' not found in dataset.
'The Midnight LibraryLife of Pi' not found in dataset.
'Handmaid's Tale' not found in 

,1507
Book-Title,Dracula
Book-Author,Bram Stoker
Publisher,Dorset Press


recs:                     Book-Title  Book-Author           Publisher Reference Book
14594                 Dracula  Bram Stoker  Barnes Noble Books        Dracula
5969   Dracula (Tor Classics)  Bram Stoker           Tor Books        Dracula
'Crime and Punishment' not found in dataset.
Book:  Misery 

Multiple entries found for 'Misery'. Using the first one:


,1480
Book-Title,Misery
Book-Author,Stephen King
Publisher,New Amer Library


recs:                     Book-Title   Book-Author           Publisher Reference Book
14594                 Dracula   Bram Stoker  Barnes Noble Books        Dracula
5969   Dracula (Tor Classics)   Bram Stoker           Tor Books        Dracula
6467                       It  Stephen King    New Amer Library         Misery
2746                   Misery  Stephen King        Viking Books         Misery
'Catcher in The Rye' not found in dataset.
Book:  The Godfather 

recs:                     Book-Title   Book-Author           Publisher Reference Book
14594                 Dracula   Bram Stoker  Barnes Noble Books        Dracula
5969   Dracula (Tor Classics)   Bram Stoker           Tor Books        Dracula
6467                       It  Stephen King    New Amer Library         Misery
2746                   Misery  Stephen King        Viking Books         Misery
6240               Fool's Die    Mario Puzo         Signet Book  The Godfather
1698             The Fourth K    Mario Puzo    Rand

,3271
Book-Title,Frankenstein
Book-Author,Mary Wollstonecraft Shelley
Publisher,Signet Book


recs:                                                Book-Title  \
14594                                            Dracula   
5969                              Dracula (Tor Classics)   
6467                                                  It   
2746                                              Misery   
6240                                          Fool's Die   
1698                                        The Fourth K   
16971                                       Frankenstein   
9380   The Essential Frankenstein: Including the Comp...   

                       Book-Author           Publisher Reference Book  
14594                  Bram Stoker  Barnes Noble Books        Dracula  
5969                   Bram Stoker           Tor Books        Dracula  
6467                  Stephen King    New Amer Library         Misery  
2746                  Stephen King        Viking Books         Misery  
6240                    Mario Puzo         Signet Book  The Godfather  
1698                

In [50]:
print("Recommendations for Friend 7:")
friend7_recs

Recommendations for Friend 7:


""


In [51]:
print("Recommendations for Friend 8:")
friend8_recs

Recommendations for Friend 8:


,Book-Title,Book-Author,Publisher,Reference Book
0,Animal Farm,George Orwell,New Amer Library Classics,1984
1,1984 (Signet Classics (Paperback)),George Orwell,Sagebrush Bound,1984
2,The Lord of the Rings,J. R. R. Tolkien,Houghton Mifflin Company,The Lord of the Rings
3,"The Two Towers (The Lord of the Rings, Part 2)",J. R. R. Tolkien,Houghton Mifflin Company,The Lord of the Rings


In [52]:
print("Recommendations for Friend 1:")
friend1_recs

Recommendations for Friend 1:


,Book-Title,Book-Author,Publisher,Reference Book
0,Dracula,Bram Stoker,Barnes Noble Books,Dracula
1,Dracula (Tor Classics),Bram Stoker,Tor Books,Dracula
2,It,Stephen King,New Amer Library,Misery
3,Misery,Stephen King,Viking Books,Misery
4,Fool's Die,Mario Puzo,Signet Book,The Godfather
5,The Fourth K,Mario Puzo,Random House Inc,The Godfather
6,Frankenstein,Mary Wollstonecraft Shelley,Barnes & Noble,Frankenstein
7,The Essential Frankenstein: Including the Comp...,Mary Wollstonecraft Shelley,Penguin USA,Frankenstein


## Content-based filter with enriched dataset

In [ ]:
# code i used to create enriched_books_dataset_smaller bc i didn't need the one hot encoded columns. now we can just get it from github

#enriched_df = pd.read_csv('enriched_books_dataset.csv')
#enriched_smaller_df = enriched_df.iloc[:, :11] # without the one hot encoding stuff
#output_file_path = '/content/enriched_books_dataset_smaller.csv'
#enriched_smaller_df.to_csv(output_file_path, index=False)

In [32]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import hstack
from sklearn.decomposition import TruncatedSVD

enriched_smaller_df = pd.read_csv('https://raw.githubusercontent.com/tamaraamicic/Honours-Project-Datasets/refs/heads/main/enriched_books_dataset_smaller.csv', sep=',', on_bad_lines='skip', index_col=False)

# list of textual features to weight and combine later
text_features = ['Book-Title', 'Book-Author', 'subjects', 'synopsis']

# for storage of all of the individual tfidf matrices
tfidf_matrices = {}
vectorizers = {}

# apply tfidf to each textual feature independently
for feature in text_features:
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(enriched_smaller_df[feature].astype(str))
    tfidf_matrices[feature] = tfidf_matrix
    vectorizers[feature] = tfidf  # store in case needed later for unseen data

# different strategies to try out (defining weight combinations)
strategies = {
    "title": {'Book-Title': 1.0},
    "author": {'Book-Author': 1.0},
    "subjects": {'subjects': 1.0},
    "synopsis": {'synopsis': 1.0},
    "title_author": {'Book-Title': 0.5, 'Book-Author': 0.5},
    "title_subjects": {'Book-Title': 0.5, 'subjects': 0.5},
    "title_synopsis": {'Book-Title': 0.5, 'synopsis': 0.5},
    "author_subjects": {'Book-Author': 0.5, 'subjects': 0.5},
    "author_synopsis": {'Book-Author': 0.5, 'synopsis': 0.5},
    "subjects_synopsis": {'subjects': 0.5, 'synopsis': 0.5},
    "title_author_subject": {'Book-Title': 0.33, 'Book-Author': 0.33, 'subjects': 0.34},
    "title_author_synopsis": {'Book-Title': 0.33, 'Book-Author': 0.33, 'synopsis': 0.34},
    "title_subjects_synopsis": {'Book-Title': 0.33, 'subjects': 0.33, 'synopsis': 0.34},
    "author_subjects_synopsis": {'Book-Author': 0.33, 'subjects': 0.33, 'synopsis': 0.34},
    "title_author_subjects_synopsis": {'Book-Title': 0.25, 'Book-Author': 0.25, 'subjects': 0.25, 'synopsis': 0.25}
}

similarity_models = {}

# get all unique languages, so that we can build one model per strategy per language
languages = enriched_smaller_df['language'].dropna().unique() # there are 23 languages in the dataset. so we create 15*23 = 345 models

for strategy_name, weights in strategies.items():
    for lang in languages:
        # subset the dataframe to books in that language
        lang_df = enriched_smaller_df[enriched_smaller_df['language'] == lang]
        if lang_df.empty:
            continue  # skip if no books in this language

        # combining tfidf matrices according to weight. essentially just concatenating them horizontally
        matrices_to_stack = []
        for feature, weight in weights.items():
            tfidf = vectorizers[feature]
            matrix = tfidf.transform(lang_df[feature].astype(str)) * weight
            matrices_to_stack.append(matrix)

        combined_matrix = hstack(matrices_to_stack)

        # fit nearest neighbors
        nn_model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6) # get top 6, then drop the first (the book itself, because it is perfectly similar to itself) to get top 5 books
        nn_model.fit(combined_matrix)

        # dictionary with keys strategy and language to save the model, combined matrix, and df to be used later
        similarity_models[(strategy_name, lang)] = {
            "model": nn_model,
            "combined_matrix": combined_matrix,
            "df": lang_df.reset_index(drop=True)  # store index-matching df
        }

In [33]:
import re

def normalize_title(title):
    title = title.lower().strip()
    title = re.sub(r'\([^)]*\)', '', title) # remove text in parentheses
    title = re.sub(r'[^a-z0-9 ]+', '', title) # remove punctuation (anything that is not a letter a-z, a digit 0-9, or a space)
    title = re.sub(r'\s+', ' ', title) # normalize whitespace (turns tabs, newlines, multiple spaces, etc. into just one space)
    return title.strip()

def improved_content_filter(title, num_recommendations, strategy_name):
    try:
        # find the book and language
        original_row = enriched_smaller_df[enriched_smaller_df['Book-Title'] == title].iloc[0]
        target_language = original_row['language']
    except IndexError:
        raise ValueError(f"Book titled '{title}' not found in dataset.")

    # find the appropriate model to use
    key = (strategy_name, target_language)
    if key not in similarity_models:
        raise ValueError(f"No model found for strategy '{strategy_name}' and language '{target_language}'.")

    model_info = similarity_models[key]
    model = model_info["model"]
    matrix = model_info["combined_matrix"]
    lang_df = model_info["df"]

    # get the index in the language-filtered DataFrame
    local_index = lang_df[lang_df['Book-Title'] == title].index[0]

    # get top N+1 neighbors and skip the book itself
    distances, indices = model.kneighbors(matrix[local_index].reshape(1, -1), n_neighbors=50 + 1)  # larger pool
    top_indices = indices[0][1:]  # skip self

    recs = lang_df.iloc[top_indices][['Book-Title', 'Book-Author', 'Publisher']].copy()

    # filter out editions/duplicates of the base book
    base_norm = normalize_title(title)
    seen_titles = set()
    filtered_recs = []

    for _, row in recs.iterrows():
        rec_title = row['Book-Title']
        norm_rec = normalize_title(rec_title)

        if base_norm in norm_rec:
            continue
        if norm_rec in seen_titles:
            continue

        seen_titles.add(norm_rec)
        filtered_recs.append(row)

        if len(filtered_recs) == num_recommendations:
            break

    return pd.DataFrame(filtered_recs)

In [ ]:
improved_content_filter("Decision in Normandy", 5, "title_author_subjects_synopsis")

,Book-Title,Book-Author,Publisher
2420,Under Blood Red Sun,Graham Salisbury,Bantam Doubleday Dell Publishing Group
6753,The Greatest Generation,TOM BROKAW,Random House
1542,"Band of Brothers : E Company, 506th Regiment, ...",Stephen E. Ambrose,Simon & Schuster
9100,Torpedo Junction: U-Boat War Off America's Eas...,Homer Hickam,Dell Publishing Company
5310,The Red Orchestra (Witnesses to War),Gilles Perrault,Random House Inc


In [35]:
improved_content_filter("El Senor De Los Anillos: LA Comunidad Del Anillo (Lord of the Rings (Spanish))", 5, "subjects")

,Book-Title,Book-Author,Publisher
26,Contact,Carl Sagan,Pocket
117,LOVE OF THE LAST TYCOON,F. Scott Fitzgerald,Scribner
2,El Senor De Los Anillos: Las DOS Torres (Lord ...,J. R. R. Tolkien,Minotauro
234,Otra Vuelta de Tuerca,Henry James,Alianza
226,El Color De La Magia,Terry Pratchett,Plaza & Janes Editor


In [4]:
def compare_strategies_binary_matrix(title, num_recommendations, strategy_names):
    from collections import defaultdict

    # dictionary mapping book titles to the strategies that recommended them
    book_to_strategies = defaultdict(set)

    for strategy in strategy_names:
        recs = improved_content_filter(title, num_recommendations, strategy)
        for book in recs['Book-Title']:
            book_to_strategies[book.strip().lower()].add(strategy)

    # get all unique book titles
    unique_books = list(book_to_strategies.keys())

    # create binary matrix
    binary_data = []
    total_strategies = len(strategy_names)

    for book in unique_books:
        row = {
            "Book-Title": book.title(),
        }
        count = 0
        for strategy in strategy_names:
            is_rec = int(strategy in book_to_strategies[book])
            row[strategy] = is_rec
            count += is_rec
        row["# Content-Based Strategies"] = count
        row["% Content-Based Strategies"] = round((count / total_strategies) * 100, 2)
        binary_data.append(row)

    return pd.DataFrame(binary_data).sort_values("# Content-Based Strategies", ascending=False)

strategy_names = [
    "title",
    "author",
    "subjects",
    "synopsis",
    "title_author",
    "title_subjects",
    "title_synopsis",
    "author_subjects",
    "author_synopsis",
    "subjects_synopsis",
    "title_author_subject",
    "title_author_synopsis",
    "title_subjects_synopsis",
    "author_subjects_synopsis",
    "title_author_subjects_synopsis"
]
comparison_binary_df = compare_strategies_binary_matrix("Decision in Normandy", 5, strategy_names)
comparison_binary_df

,Book-Title,title,author,subjects,synopsis,title_author,title_subjects,title_synopsis,author_subjects,author_synopsis,subjects_synopsis,title_author_subject,title_author_synopsis,title_subjects_synopsis,author_subjects_synopsis,title_author_subjects_synopsis,# Content-Based Strategies,% Content-Based Strategies
10,Under Blood Red Sun,0,0,1,0,0,1,0,1,0,1,1,0,1,1,1,8,53.33
3,"Band Of Brothers : E Company, 506Th Regiment, ...",1,0,0,0,1,1,1,0,0,0,1,1,1,0,1,8,53.33
13,The Greatest Generation,0,0,1,0,0,1,0,1,0,1,1,0,1,1,1,8,53.33
12,The Red Orchestra (Witnesses To War),0,0,1,0,0,1,0,1,0,1,1,0,0,1,1,7,46.67
11,Battle Of Midway Island (The Great Battles Of ...,0,0,1,0,0,1,0,1,0,1,1,0,1,1,0,7,46.67
14,Torpedo Junction: U-Boat War Off America'S Eas...,0,0,1,0,0,0,0,1,0,1,0,0,1,1,1,6,40.00
1,Your Sacred Self: Making The Decision To Be Free,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,4,26.67
0,Decision At Doona,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,4,26.67
5,The Eleventh Summer,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,4,26.67
2,Decision In Philadelphia: The Constitutional C...,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,4,26.67


## Collaborative Methods

### LightFM with Book Ratings

In [4]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp311-cp311-linux_x86_64.whl size=831161 sha256=d58c682c2ec53d7b7939603d1a2e582719e0ae098b48fa8c969ad3cf93a0dfb0
  Stored in directory: /root/.cache/pip/wheels/b9/0d/8a/0729d2e6e3ca2a898ba55201f905da7db3f838a33df5b3fcdd
Successfully built lightfm


In [5]:
#https://medium.com/@dikosaktiprabowo/hybrid-recommendation-system-using-lightfm-e10dd6b42923
#https://making.lyst.com/lightfm/docs/
#https://medium.com/@murattyldrm7/building-a-recommendation-system-with-lightfm-35394c8d90fb

import pandas as pd
import numpy as np
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k

book_ratings_df = pd.read_csv('https://raw.githubusercontent.com/tamaraamicic/Honours-Project-Datasets/refs/heads/main/book_ratings.dat', sep='\t', on_bad_lines='skip', index_col=False)
enriched_smaller_df = pd.read_csv('https://raw.githubusercontent.com/tamaraamicic/Honours-Project-Datasets/refs/heads/main/enriched_books_dataset_smaller.csv', sep=',', on_bad_lines='skip', index_col=False)

# only keep ratings for books that exist in the enriched dataset
valid_books = set(enriched_smaller_df['Book_ID'])
book_ratings_df = book_ratings_df[book_ratings_df['item'].isin(valid_books)].copy()

# converting ratings into binary interactions (1 = liked, 0 = not liked) because LightFM works best with implicit feedback (interaction vs. no interaction)
book_ratings_df['rating'] = book_ratings_df['rating'].apply(lambda x: 1 if x > 5 else 0)

lightfm_ratings_dataset = Dataset() # LightFM dataset object

# creating list of words (that comprise of the book title, author, subjects, and synopsis)
# LightFM will learn that books with similar words in their title, author, subjects, or synopsis are related
enriched_smaller_df['item_features'] = (
    enriched_smaller_df['Book-Title'].astype(str) + " " +
    enriched_smaller_df['Book-Author'].astype(str) + " " +
    enriched_smaller_df['subjects'].astype(str) + " " +
    enriched_smaller_df['synopsis'].astype(str)
).apply(lambda x: x.split())  # split into list of words

# Ensure we pass the same features to fit() and build_item_features()
unique_features = set(enriched_smaller_df['item_features'].explode().unique()) # getting all the unique words among all book titles, authors, subject, and synposis

# fits/registersr dataset with users, items, and item features. this creates internal mappings so that LightFM can process these entities correctly later
lightfm_ratings_dataset.fit(
    users=book_ratings_df['user'].unique(),
    items=enriched_smaller_df['Book_ID'].unique(), # using all books (not just rated ones) so that books that have not been rated could also be recommended
    item_features=unique_features
)

# building a sparse user-book interaction matrix internally
# rows = users
# columns = books
# values = 1 (liked) or 0 (not liked)
(lightfm_ratings_interactions, lightfm_ratings_weights) = lightfm_ratings_dataset.build_interactions([
    (row['user'], row['item'], row['rating'])
    for _, row in book_ratings_df.iterrows()
])

# internally creates a sparse item feature matrix
# rows = book ids
# columns = features
# values = 1 if the book has the feature, 0 if not
# but instead of storing a huge matrix with mostly zeros, LightFM only stores non-zero values for efficiency
lightfm_ratings_item_features = lightfm_ratings_dataset.build_item_features([
    (row['Book_ID'], row['item_features'])
    for _, row in enriched_smaller_df.iterrows()
])


In [6]:
# Train a hybrid model (both collaborative & content-based filtering)
lightfm_ratings_model = LightFM(loss='warp') # weighted approximate-rank pairwise loss function
                             # optimizes for ranking quality instead of prediction accuracy
                             # instead of predicting a rating (like 1-10), WARP ranks books based on likelihood of interest.
                             # works better for implicit feedback
                             # prioritizes learning what books should be ranked higher for each user

# learns patterns from both user ratings & book metadata.
lightfm_ratings_model.fit(lightfm_ratings_interactions, item_features=lightfm_ratings_item_features, epochs=10, num_threads=4) # more epochs = better learning, but too many can cause overfitting
                                                                               # 4 cpu threads to speed up training

In [7]:
# we do hybrid, even though this is one of our collaborative methods, bc the book content helps the LightFM model work better (for cold start).
# the model is still largely collaborative
def recommend_books_hybrid_df(model, dataset, item_features_to_use, user_id, n=10):
    item_ids = np.arange(len(dataset.mapping()[2])) # get all book indices

    scores = model.predict(user_id, item_ids, item_features=item_features_to_use) # predicting scores

    top_items = np.argsort(-scores)[:n*3]  # sorts books highest to lowest, gets top 3*n (get more than needed, in case we remove duplicates later)

    # LightFM internally uses numeric indices for books. this converts the internal book indices back to original book ids
    item_map = {v: k for k, v in dataset.mapping()[2].items()} # dataset.mapping()[2] is item (book) mapping
    recommended_book_ids = [item_map[i] for i in top_items]

    # getting book details from enriched_smaller_df
    recommended_books_df = enriched_smaller_df[enriched_smaller_df['Book_ID'].isin(recommended_book_ids)].copy()

    # add predicted ratings
    predicted_scores = {item_map[i]: scores[i] for i in top_items}
    recommended_books_df.loc[:, 'Predicted Rating'] = recommended_books_df['Book_ID'].map(predicted_scores)

    recommended_books_df = recommended_books_df.sort_values('Predicted Rating', ascending=False)

    # drop duplicates and get top n
    recommended_books_df = recommended_books_df.drop_duplicates(subset=['Book-Title', 'Book-Author'])
    recommended_books_df = recommended_books_df.head(n)

    return recommended_books_df[['Book-Title', 'Book-Author', 'Publisher', 'Predicted Rating']]

user_id = 1
recommended_books_df = recommend_books_hybrid_df(lightfm_ratings_model, lightfm_ratings_dataset, lightfm_ratings_item_features, user_id)
recommended_books_df

,Book-Title,Book-Author,Publisher,Predicted Rating
406,The Rescue,Nicholas Sparks,Warner Books,0.408025
1887,Desperation,Stephen King,Signet Book,0.296132
227,Impossible Vacation,Spalding Gray,Random House Inc,0.080204
1179,Dolores Claiborne,Stephen King,Signet Book,0.071112
2331,Reckless Abandon,Stuart Woods,Putnam Publishing Group,0.029769
2753,Deep Water Passage: A Spiritual Journey at Mid...,Ann Linnea,Little Brown & Co,-0.031826
7,Airframe,Michael Crichton,Ballantine Books,-0.047693
798,Mansfield Park (Signet Classics (Paperback)),Jane Austen,Signet Book,-0.134479
8264,Loving,Danielle Steel,Dell,-0.168642
2745,Misery,Stephen King,Viking Books,-0.192507


### LightFM with Book Access History

In [8]:
#https://medium.com/@dikosaktiprabowo/hybrid-recommendation-system-using-lightfm-e10dd6b42923
#https://making.lyst.com/lightfm/docs/
#https://medium.com/@murattyldrm7/building-a-recommendation-system-with-lightfm-35394c8d90fb

import pandas as pd
import numpy as np
from lightfm import LightFM
from lightfm.data import Dataset

# Load datasets
book_history_df = pd.read_csv('https://raw.githubusercontent.com/tamaraamicic/Honours-Project-Datasets/refs/heads/main/book_history.dat', sep='\t', on_bad_lines='skip', index_col=False)
enriched_smaller_df = pd.read_csv('https://raw.githubusercontent.com/tamaraamicic/Honours-Project-Datasets/refs/heads/main/enriched_books_dataset_smaller.csv', sep=',', on_bad_lines='skip', index_col=False)

# only keep ratings for books that exist in the enriched dataset
valid_books = set(enriched_smaller_df['Book_ID'])
book_history_df = book_history_df[book_history_df['item'].isin(valid_books)].copy()


# creating list of words (that comprise of the book title, author, subjects and synopsis)
# LightFM will learn that books with similar words in their title, author, subjects or synopsis are related
# could extend this when we combine with the new dataset
enriched_smaller_df['item_features'] = (
    enriched_smaller_df['Book-Title'].astype(str) + " " +
    enriched_smaller_df['Book-Author'].astype(str) + " " +
    enriched_smaller_df['subjects'].astype(str) + " " +
    enriched_smaller_df['synopsis'].astype(str)
).apply(lambda x: x.split())

# Ensure we pass the same features to fit() and build_item_features()
unique_features = set(enriched_smaller_df['item_features'].explode().unique()) # getting all the unique words among all book titles and authors

lightfm_history_dataset = Dataset() # LightFM dataset object

# fits/registersr dataset with users, items, and item features. this creates internal mappings so that LightFM can process these entities correctly later
lightfm_history_dataset.fit(
    users=book_history_df['user'].unique(),
    items=book_history_df['item'].unique(),
    item_features=unique_features
)

# building a sparse user-book interaction matrix internally
# rows = users
# columns = books
# values = 1 (liked)
(lightfm_history_interactions, lightfm_history_weights) = lightfm_history_dataset.build_interactions([
    (row['user'], row['item'], 1) for _, row in book_history_df.iterrows() # always 1 since all interactions are access events
])

# internally creates a sparse item feature matrix
# rows = book ids
# columns = features
# values = 1 if the book has the feature, 0 if not
# but instead of storing a huge matrix with mostly zeros, LightFM only stores non-zero values for efficiency
lightfm_history_item_features = lightfm_history_dataset.build_item_features([
    (row['Book_ID'], row['item_features']) for _, row in enriched_smaller_df.iterrows()
])


In [9]:
# Train a hybrid model (both collaborative & content-based filtering)
lightfm_history_model = LightFM(loss='warp') # weighted approximate-rank pairwise loss function
                             # optimizes for ranking quality instead of prediction accuracy
                             # instead of predicting a rating (like 1-10), WARP ranks books based on likelihood of interest.
                             # works better for implicit feedback
                             # prioritizes learning what books should be ranked higher for each user

# learns patterns from both user ratings & book metadata.
lightfm_history_model.fit(lightfm_history_interactions, item_features=lightfm_history_item_features, epochs=10, num_threads=4) # more epochs = better learning, but too many can cause overfitting
                                                                               # 4 cpu threads to speed up training

In [10]:
user_id = 1
recommended_books_df = recommend_books_hybrid_df(lightfm_history_model, lightfm_history_dataset, lightfm_history_item_features, user_id) # reusing the same function as before but calling it with the new model (lightfm_history_model)
recommended_books_df

,Book-Title,Book-Author,Publisher,Predicted Rating
406,The Rescue,Nicholas Sparks,Warner Books,0.578710
227,Impossible Vacation,Spalding Gray,Random House Inc,0.510117
1017,Life's Little Instruction Book Volume II,H. Jackson Brown,Thomas Nelson Inc,0.498149
2155,Winterdance: The Fine Madness of Running the I...,Gary Paulsen,Harvest Books,0.481193
642,Imaginary Lands,Robin McKinley,Ace Books,0.465427
503,The Truth About Texas: Who Needs to Brag? We'V...,Anne Dingus,Gulf Publishing Company,0.459152
1135,"Tycoon'S Temptation (Silhouette Desire, No. 1414)",Katherine Garbera,Silhouette,0.449856
288,Biblioteca Universale Rizzoli: Sulla Sponda De...,P Coelho,Fabbri - RCS Libri,0.342818
1271,Airframe,Michael Crichton,Alfred A. Knopf,0.338503
798,Mansfield Park (Signet Classics (Paperback)),Jane Austen,Signet Book,0.308242


### SVD

In [14]:
!pip install numpy==1.24.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 15.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
pymc 5.22.0 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is incompatible.
blosc2 3.3.1 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is 

In [11]:
!pip install scikit-surprise
#!pip install "numpy<2.0" --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505212 sha256=6bf6cf6c7727c3c924f96bb3078672755e7bab5e3dcd80b77bf05a94c319e860
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


Note: Even though the book_ratings table looks dense (fully populated with ratings), it is actually just a list of interactions. The real user-book matrix (if we tried to create it) would be very large and sparse.

We have:
- 1295 users
- 14,684 books
- 62,657 ratings recorded

Then the full user-book matrix would be 1,295 × 14,684 = 19,015,780 cells, but we only have 62,657 known ratings (only 0.3295% of the matrix is filled).

Since the matrix is sparse, we make use of singular value decomposition (SVD). SVD transforms this sparse matrix into two matrices: Users x Features and Features x Books. Multiplied we get Users x Books.

In [11]:
import pandas as pd
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

# https://medium.com/@mariomg85/leveraging-surprise-library-for-recommender-systems-in-python-915e699f7a99
# https://surprise.readthedocs.io/en/stable/reader.html

book_ratings_df = pd.read_csv('https://raw.githubusercontent.com/tamaraamicic/Honours-Project-Datasets/refs/heads/main/book_ratings.dat', sep='\t', on_bad_lines='skip', index_col=False)

reader = Reader(rating_scale=(1, 10)) # defining that the rating scale is from 1-10
data = Dataset.load_from_df(book_ratings_df[['user', 'item', 'rating']], reader) # format for Surprise
trainset, testset = train_test_split(data, test_size=0.2, random_state=42) # 80% train, 20% test

svd_model = SVD(n_factors=50, random_state=42) # training. we define 50 latent factors -- factors that are hidden, not directly observed. for example genres, writing styles, book popularity, but we don’t explicitly define them. instead, SVD discovers these factors automatically
                                               # i tried changing this around though and it didn't change the value much...
svd_model.fit(trainset)
predictions = svd_model.test(testset) # using trained SVD model to make
rmse = accuracy.rmse(predictions) # root MSE is preferred over MSE in recommender systems

RMSE: 1.4810


In [ ]:
#!pip install numpy==1.24.4

In [12]:
items_info_cleaned_2_df = pd.read_csv('https://raw.githubusercontent.com/tamaraamicic/Honours-Project-Datasets/refs/heads/main/items_info_cleaned_2.dat', sep='\t', on_bad_lines='skip', index_col=False,)

all_books = set(items_info_cleaned_2_df['Book_ID'])

def get_user_rated_books(user_id): # books that a user has rated
    return set(book_ratings_df[book_ratings_df['user'] == user_id]['item'])

def get_unrated_books(user_id): # books that a user hasn't rated
    rated_books = get_user_rated_books(user_id)
    return list(all_books - rated_books)

def predict_ratings_for_user(user_id): # predicts user rating for all books that the user hasn't rated
    unseen_books = get_unrated_books(user_id)
    predictions = [(book, svd_model.predict(user_id, book).est) for book in unseen_books]
    return sorted(predictions, key=lambda x: x[1], reverse=True)  # sorts books from highest to lowest rating

# Function to recommend top-N books
def recommend_books_SVD(user_id, n=10):
    top_books = predict_ratings_for_user(user_id)[:n]  # gets top N books

    book_ids = []
    predicted_scores = {}

    # to get the predicted rating
    for book_id, predicted_rating in top_books:
        book_ids.append(book_id)
        predicted_scores[book_id] = predicted_rating

    recommended_books_SVD = items_info_cleaned_2_df[items_info_cleaned_2_df['Book_ID'].isin(book_ids)].copy()
    recommended_books_SVD.loc[:, 'Predicted Rating'] = recommended_books_SVD['Book_ID'].map(predicted_scores) # adding the predicted rating

    return recommended_books_SVD[['Book-Title', 'Book-Author', 'Publisher', 'Predicted Rating']]


user_id = 1
recommended_books_SVD = recommend_books_SVD(user_id)
recommended_books_SVD

,Book-Title,Book-Author,Publisher,Predicted Rating
13,Seabiscuit: An American Legend,LAURA HILLENBRAND,Ballantine Books,8.790367
432,All of Me: A Voluptuous Tale,Venise T. Berry,Dutton Books,8.832133
703,Lonesome Dove,Larry McMurtry,Pocket,8.893103
853,Watership Down,Richard George Adams,Harpercollins,8.802692
1302,Mirror Image,Danielle Steel,Dell Publishing Company,8.802787
2057,"Tucket's Gold (Tuckets Adventures, Book 4)",Gary Paulsen,Yearling Books,8.925297
3151,To Love This Life: Quotations From Helen Keller,Helen Keller,Scholastic Paperbacks,8.841983
3189,A Common Life : The Wedding Story (The Mitford...,Jan Karon,Viking Books,8.862984
7438,The Broken Promise Land,Marcia Muller,Warner Books,8.907912
10307,Coming Back: A Psychiatrist Explores Past-Life...,Raymond A. Moody,Bantam,8.808836


### Similarity

In [13]:
import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

book_ratings_df = pd.read_csv('https://raw.githubusercontent.com/tamaraamicic/Honours-Project-Datasets/refs/heads/main/book_ratings.dat', sep='\t', on_bad_lines='skip', index_col=False)

user_item_matrix = book_ratings_df.pivot(index='user', columns='item', values='rating').fillna(0)

user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

def get_similar_users(user_id, n=5):
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:n+1]  # skips itself by skipping the first value at index 0 (perfect similarity of 1)
    return similar_users.index.tolist()

def recommend_books_user_based(user_id, n=10):
    similar_users = get_similar_users(user_id, n=10)  # find top 10 similar users according to similarity score

    similar_users_books = book_ratings_df[book_ratings_df['user'].isin(similar_users)] # books rated by similar users
    rated_books = set(book_ratings_df[book_ratings_df['user'] == user_id]['item'])  # books rated by target user
    unseen_books = similar_users_books[~similar_users_books['item'].isin(rated_books)]  # books rated by similar users NOT seen by target user

    recommended_books = unseen_books.groupby('item')['rating'].mean().sort_values(ascending=False).head(n) # lists top n recommendations, printing the average rating (of ratings from the similar users)

    # convert to dictionary with book metadata
    pred_dict = recommended_books.to_dict()
    top_book_ids = recommended_books.index.tolist()
    items_info_cleaned_2_df = pd.read_csv('https://raw.githubusercontent.com/tamaraamicic/Honours-Project-Datasets/refs/heads/main/items_info_cleaned_2.dat', sep='\t', on_bad_lines='skip', index_col=False)
    recommended_books_df = items_info_cleaned_2_df[items_info_cleaned_2_df['Book_ID'].isin(pred_dict.keys())].copy()
    recommended_books_df['Predicted Rating'] = recommended_books_df['Book_ID'].map(pred_dict)

    # sorting
    recommended_books_df['SortOrder'] = recommended_books_df['Book_ID'].apply(lambda x: top_book_ids.index(x))
    recommended_books_df = recommended_books_df.sort_values('SortOrder').drop(columns='SortOrder')

    return recommended_books_df[['Book-Title', 'Book-Author', 'Publisher', 'Predicted Rating']]

user_id = 1
recommended_books = recommend_books_user_based(user_id)
recommended_books

,Book-Title,Book-Author,Publisher,Predicted Rating
5402,The Fellowship of the Ring (The Lord of the Ri...,J. R. R. Tolkien,Houghton Mifflin Company,10.0
2131,The Evil That Men Do (Buffy the Vampire Slayer),Nancy Holder,Simon Spotlight Entertainment,10.0
8235,Stimmts? Noch mehr moderne Legenden im Test.,Christoph DrÃ?Â¶sser,Rowohlt Tb.,10.0
8178,Amelia Bedelia (I Can Read),Peggy Parish,HarperTrophy,10.0
8163,Hornet Flight,Ken Follett,Dutton Books,10.0
8104,Blood Work,Michael Connelly,Warner Books,10.0
2083,Flesh and Blood,Jonathan Kellerman,Ballantine Books,10.0
8066,Nothing But the Night,Bill Pronzini,Walker & Company,10.0
7998,Singing Songs,Meg Tilly,Penguin USA,10.0
2492,The Tenth Justice,Brad Meltzer,Warner Books,10.0


### KNN

In [14]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# -------------------------------
# 1. Load the Ratings Data
# -------------------------------
book_ratings_df = pd.read_csv(
    'https://raw.githubusercontent.com/tamaraamicic/Honours-Project-Datasets/refs/heads/main/book_ratings.dat',
    sep='\t',
    on_bad_lines='skip',
    index_col=False
)

# -------------------------------
# 2. Build the User-Item Ratings Matrix
# -------------------------------
# Pivot the ratings table so that rows are users and columns are items (books)
ratings_pivot = book_ratings_df.pivot(index='user', columns='item', values='rating')
# For computing cosine similarity, fill missing values with 0.
ratings_filled = ratings_pivot.fillna(0)

# -------------------------------
# 3. Compute User Similarity Matrix
# -------------------------------
# Use cosine similarity from scikit-learn
user_similarity = cosine_similarity(ratings_filled)
# Create a DataFrame for easier lookup; its rows/columns are the user IDs
user_similarity_df = pd.DataFrame(user_similarity, index=ratings_filled.index, columns=ratings_filled.index)

# Create a mapping from user ID to index
user_id_to_index = {user: idx for idx, user in enumerate(ratings_filled.index)}

# -------------------------------
# 4. Define Prediction Functions
# -------------------------------
def predict_rating(user_id, book_id, k=5):
    """
    Predict rating for a given user and book using weighted average of top k similar users.
    """
    # If the book is not in the ratings pivot, we cannot predict (return NaN)
    if book_id not in ratings_pivot.columns:
        return np.nan

    # Get the target user's row index and similarity scores
    user_idx = user_id  # The index here is the user id itself from the pivot table
    # Get similarities between the target user and all other users (excluding self by setting to 0)
    sims = user_similarity_df.loc[user_id].copy()
    sims[user_id] = 0.0  # zero-out self-similarity

    # Consider only users that rated the book (i.e. non-zero rating in the original pivot)
    # ratings_filled for the given book returns a Series with index = user id
    book_ratings = ratings_filled[book_id]
    rated_mask = book_ratings > 0
    # If no other user has rated the book, return NaN (or you could use the user's average)
    if rated_mask.sum() == 0:
        return np.nan

    # Filter the similarities and corresponding ratings to those users who rated the book
    sims = sims[rated_mask]
    neighbor_ratings = book_ratings[rated_mask]

    # If k is specified, pick the top k similar users
    if k is not None and len(sims) > k:
        top_k_idx = sims.nlargest(k).index
        sims = sims.loc[top_k_idx]
        neighbor_ratings = neighbor_ratings.loc[top_k_idx]

    # Avoid division by zero; if sum of similarities is 0, return the neighbor average
    sim_sum = sims.sum()
    if sim_sum == 0:
        return neighbor_ratings.mean()
    predicted = np.dot(sims, neighbor_ratings) / sim_sum
    return predicted


def get_user_rated_books(user_id):
    """
    Return a set of book IDs that the user has rated.
    """
    return set(book_ratings_df[book_ratings_df['user'] == user_id]['item'])


def get_unrated_books(user_id):
    """
    Return a list of book IDs that the user hasn't rated.
    """
    # Load the complete set of book IDs from the items info dataset
    items_info_cleaned_2_df = pd.read_csv(
        'https://raw.githubusercontent.com/tamaraamicic/Honours-Project-Datasets/refs/heads/main/items_info_cleaned_2.dat',
        sep='\t',
        on_bad_lines='skip',
        index_col=False
    )
    all_books = set(items_info_cleaned_2_df['Book_ID'])
    rated_books = get_user_rated_books(user_id)
    return list(all_books - rated_books)


def predict_ratings_for_user(user_id, k=5):
    """
    Predict the ratings for all unseen books for a given user.
    """
    unseen_books = get_unrated_books(user_id)
    predictions = []
    for book in unseen_books:
        pred = predict_rating(user_id, book, k)
        # Only include predictions if we have a value (i.e., at least one similar neighbor rated the book)
        if not np.isnan(pred):
            predictions.append((book, pred))
    # Sort predictions from highest to lowest predicted rating
    return sorted(predictions, key=lambda x: x[1], reverse=True)


def recommend_books_KNN_custom(user_id, n=10, k=5):
    """
    Recommend top-N books for a given user using the KNN-based CF.
    """
    # Predict ratings for unseen books
    preds = predict_ratings_for_user(user_id, k=k)
    top_preds = preds[:n]

    # Build a dictionary of book: predicted rating
    pred_dict = {book: rating for book, rating in top_preds}

    # Load the items info dataset to get book details
    items_info_cleaned_2_df = pd.read_csv(
        'https://raw.githubusercontent.com/tamaraamicic/Honours-Project-Datasets/refs/heads/main/items_info_cleaned_2.dat',
        sep='\t',
        on_bad_lines='skip',
        index_col=False
    )

    # Filter items dataset for the recommended books and map the predicted ratings
    recommended_books = items_info_cleaned_2_df[items_info_cleaned_2_df['Book_ID'].isin(list(pred_dict.keys()))].copy()
    recommended_books.loc[:, 'Predicted Rating'] = recommended_books['Book_ID'].map(pred_dict)

    return recommended_books[['Book-Title', 'Book-Author', 'Publisher', 'Predicted Rating']]

# -------------------------------
# 5. Get Recommendations for a Specific User
# -------------------------------
user_id = 1
recommended_books_custom = recommend_books_KNN_custom(user_id, n=10, k=5)
print(recommended_books_custom)

                                             Book-Title        Book-Author  \
76    Twenty Minute Retreats: Revive Your Spirits in...      Rachel Harris   
331                                Your Erroneous Zones      Wayne W. Dyer   
1029                          Instances of the Number 3     Salley Vickers   
1133                                              Crows  Charles Dickinson   
1227                   You're Not a Kid Anymore When...     Jeff Foxworthy   
2177  A RIVER RUNS THROUGH IT AND OTHER STORIES : AN...     Norman Maclean   
2545                           The Once and Future King        T. H. White   
2762                                   Bless Me, Ultima      Rudolfo Anaya   
3045                                 101 Dinosaur Jokes        Phil Hirsch   
3144                               Orlando: A Biography     Virginia Woolf   

                      Publisher  Predicted Rating  
76                Pan Macmillan              10.0  
331               Harpercollins      

### Baseline Collaborative Filtering
(check notes to write about it in report - Coralie)

In [15]:
def predict_baseline(user_id, book_id):
    """
    Predict rating using Baseline CF: global mean + user bias + item bias.
    Returns np.nan if user or item is missing.
    """
    if book_id not in ratings_pivot.columns or user_id not in ratings_pivot.index:
        return np.nan

    # Global average rating (μ)
    global_mean = book_ratings_df['rating'].mean()

    # User bias (b_u)
    user_avg = ratings_pivot.loc[user_id].mean()
    user_bias = user_avg - global_mean

    # Item bias (b_i)
    item_avg = ratings_pivot[book_id].mean()
    item_bias = item_avg - global_mean

    # Predicted rating
    return global_mean + user_bias + item_bias

def recommend_books_baseline(user_id, n=10):
    """
    Recommend top-N books for a user based on Baseline CF.
    """
    unseen_books = get_unrated_books(user_id)
    predictions = []

    for book in unseen_books:
        pred = predict_baseline(user_id, book)
        if not np.isnan(pred):
            predictions.append((book, pred))

    predictions = sorted(predictions, key=lambda x: x[1], reverse=True)[:n]
    pred_dict = {book: rating for book, rating in predictions}

    # Load book metadata
    items_info_cleaned_2_df = pd.read_csv(
        'https://raw.githubusercontent.com/tamaraamicic/Honours-Project-Datasets/refs/heads/main/items_info_cleaned_2.dat',
        sep='\t',
        on_bad_lines='skip',
        index_col=False
    )

    # Get recommended book details
    recommended_books = items_info_cleaned_2_df[items_info_cleaned_2_df['Book_ID'].isin(pred_dict.keys())].copy()
    recommended_books['Predicted Rating'] = recommended_books['Book_ID'].map(pred_dict)

    return recommended_books[['Book-Title', 'Book-Author', 'Publisher', 'Predicted Rating']]

# Test it with a user
user_id = 1
recommended_baseline_books = recommend_books_baseline(user_id, n=10)
print(recommended_baseline_books)


                                            Book-Title         Book-Author  \
6    If I'd Known Then What I Know Now: Why Not Lea...       J. R. Parrish   
25                                  All the King's Men  Robert Penn Warren   
78                                                Hush        Anne Frasier   
110                                Memoirs of a Geisha       Arthur Golden   
134   Postmortem (Kay Scarpetta Mysteries (Paperback))   Patricia Cornwell   
156                                Moonheart (Newford)     Charles de Lint   
318                  The Teacher from the Black Lagoon         Mike Thaler   
331                               Your Erroneous Zones       Wayne W. Dyer   
395                                    Waking the Dead       Scott Spencer   
403  Die Grenzen des Geschlechts: Anleitungen zum S...       Cheryl Benard   

                    Publisher  Predicted Rating  
6               Cypress House           10.1088  
25              Harvest Books           1

## Cold Start Implementation
If we have a new user that has no ratings, we give suggestions based on books that are popular among that age group and geo location.

In [16]:
import pandas as pd
import numpy as np

# -------------------------------
# 1. Load and Preprocess the User Info Data
# -------------------------------
user_info_url = 'https://raw.githubusercontent.com/tamaraamicic/Honours-Project-Datasets/refs/heads/main/users_info.dat'
user_info_df = pd.read_csv(user_info_url, sep='\t', on_bad_lines='skip', index_col=False)

# Standardize location strings to lower-case and remove any leading/trailing whitespace
user_info_df['Location'] = user_info_df['Location'].str.lower().str.strip()
# Ensure Age is numeric (if necessary, convert or handle missing ages)
user_info_df['Age'] = pd.to_numeric(user_info_df['Age'], errors='coerce')

# -------------------------------
# 2. Define the Cold-Start Recommendation Function
# -------------------------------
def recommend_books_cold_start(new_age, new_location, n=10, age_threshold=5):
    """
    Recommend books for a new user based solely on demographic data (Age and Location).

    Parameters:
      new_age (int): Age of the new user.
      new_location (str): Location string of the new user (e.g., "minneapolis, minnesota, usa").
      n (int): Number of recommendations desired.
      age_threshold (int): Acceptable difference in age for considering users as similar.

    Returns:
      DataFrame: A dataframe containing the top-N recommended books with their title, author, publisher,
                 and the average rating from similar users.
    """
    # Convert new user's location to lower-case for matching
    new_location_lower = new_location.lower().strip()

    # Filter users within the age threshold
    similar_users = user_info_df[ (user_info_df['Age'].notnull()) &
                                  (abs(user_info_df['Age'] - new_age) <= age_threshold) ]

    # Further filter by location - here we keep users whose Location string contains the new user's location.
    # This is a simple heuristic; we might use more sophisticated location matching.
    similar_users = similar_users[ similar_users['Location'].str.contains(new_location_lower, na=False) ]

    # If no users match location criteria, relax the condition to use age only.
    if similar_users.empty:
        similar_users = user_info_df[ (user_info_df['Age'].notnull()) &
                                      (abs(user_info_df['Age'] - new_age) <= age_threshold) ]

    if similar_users.empty:
        print("No similar users found based on the provided demographics.")
        return pd.DataFrame()  # or you could return overall popular books

    similar_user_ids = similar_users['User-ID'].unique()

    # -------------------------------
    # 3. Load the Book Ratings Data and Filter by Similar Users
    # -------------------------------
    book_ratings_url = 'https://raw.githubusercontent.com/tamaraamicic/Honours-Project-Datasets/refs/heads/main/book_ratings.dat'
    book_ratings_df = pd.read_csv(book_ratings_url, sep='\t', on_bad_lines='skip', index_col=False)

    # Filter ratings to those from similar users only.
    similar_ratings = book_ratings_df[ book_ratings_df['user'].isin(similar_user_ids) ]

    # In case there are no ratings from similar users, fall back on overall averages.
    if similar_ratings.empty:
        print("No ratings found from similar users. Falling back to overall average ratings.")
        similar_ratings = book_ratings_df.copy()

    # -------------------------------
    # 4. Aggregate Ratings and Get Top Books
    # -------------------------------
    # Compute the average rating per book among similar users
    book_avg_ratings = similar_ratings.groupby('item')['rating'].mean().reset_index()

    # Sort by the average rating (highest first) and pick top-N recommendations
    top_books = book_avg_ratings.sort_values('rating', ascending=False).head(n)

    # -------------------------------
    # 5. Join with Book Metadata
    # -------------------------------
    items_info_url = 'https://raw.githubusercontent.com/tamaraamicic/Honours-Project-Datasets/refs/heads/main/items_info_cleaned_2.dat'
    items_info_df = pd.read_csv(items_info_url, sep='\t', on_bad_lines='skip', index_col=False)

    # Merge to get details like Book Title, Book Author, and Publisher for when we print it out
    recommendations = items_info_df.merge(top_books, left_on='Book_ID', right_on='item')
    recommendations = recommendations.rename(columns={'rating': 'Average Rating'})

    return recommendations[['Book-Title', 'Book-Author', 'Publisher', 'Average Rating']]

# -------------------------------
# 6. Example Usage of the Cold-Start Function
# -------------------------------
# For a new user (with no prior ratings), provide age and location.
new_user_age = 25
new_user_location = "minneapolis, minnesota, usa"

recommended_books_cold_start = recommend_books_cold_start(new_user_age, new_user_location, n=10, age_threshold=5)
print(recommended_books_cold_start)

                                          Book-Title          Book-Author  \
0                                Of Love and Shadows       Isabel Allende   
1  Just Here Trying to Save a Few Lives : Tales o...          Pamela Grim   
2  Tycoon'S Temptation (Silhouette Desire, No. 1414)    Katherine Garbera   
3                                   Deadly Decisions         Kathy Reichs   
4  Nobody's Perfect: Selected Writings from the N...         Anthony Lane   
5  Past Lives, Present Dreams: How to Use Reincar...          Denise Linn   
6  A Kiss of Shadows (Meredith Gentry Novels (Pap...  LAURELL K. HAMILTON   
7               Sideways Stories from Wayside School         Louis Sachar   
8                                La fiesta del chivo   Mario Vargas Llosa   
9                                     Into the Green      Charles de Lint   

               Publisher  Average Rating  
0           Bantam Books            10.0  
1           Warner Books            10.0  
2             Silhouett

## Hybrid Recommender System

In [17]:
import pandas as pd
# THIS IS NEEDED FOR ONE OF THE METHODS, COLD-START/AGE-LOCATION METHOD

# Load the users_info dataset from the URL
user_info_url = 'https://raw.githubusercontent.com/tamaraamicic/Honours-Project-Datasets/refs/heads/main/users_info.dat'
user_info_df = pd.read_csv(user_info_url, sep='\t', on_bad_lines='skip', index_col=False)

# Example function to get a user's age and location by user_id
def get_user_demographics(user_id):
    user_row = user_info_df[user_info_df['User-ID'] == user_id]
    if not user_row.empty:
        # Retrieve the age and location of the user
        user_age = user_row['Age'].iloc[0]
        user_location = user_row['Location'].iloc[0]
        return user_age, user_location
    else:
        print(f"User {user_id} not found.")
        return None, None

# Example: Get demographics for user_id = 1
user_id = 1
user_age, user_location = get_user_demographics(user_id)
print(f"User {user_id}'s Age: {user_age}, Location: {user_location}")


User 1's Age: 24, Location: minneapolis, minnesota, usa


In [30]:
from collections import defaultdict
import pandas as pd

book_ratings_df = pd.read_csv('https://raw.githubusercontent.com/tamaraamicic/Honours-Project-Datasets/refs/heads/main/book_ratings.dat', sep='\t', on_bad_lines='skip', index_col=False)

def compare_user_top_books_binary_matrix(user_id, num_top_books=3, num_recommendations=10):
    content_strategy_names = [
        "title",
        "author",
        "subjects",
        "synopsis",
        "title_author",
        "title_subjects",
        "title_synopsis",
        "author_subjects",
        "author_synopsis",
        "subjects_synopsis",
        "title_author_subject",
        "title_author_synopsis",
        "title_subjects_synopsis",
        "author_subjects_synopsis",
        "title_author_subjects_synopsis"
    ]

    # Here, Age-Location uses the cold-start implementation
    collaborative_strategy_names = ["SVD", "KNN", "BASELINE", "AGE-LOCATION", "LIGHTFM-RATINGS", "LIGHTFM-HISTORY", "SIMILARITY"]  # Add more here and add "SVD" back here

    # get user’s top N rated books
    top_rated_books = (
        book_ratings_df[book_ratings_df['user'] == user_id]
        .sort_values(by='rating', ascending=False)
        .head(num_top_books)
    )

    # merge with enriched dataset to get book titles
    top_books_info = top_rated_books.merge(
        enriched_smaller_df,
        left_on='item',
        right_on='Book_ID',
        how='inner'
    )[['Book-Title']].drop_duplicates().reset_index(drop=True)

    # run recommendations and track which strategies recommend each book
    content_based_book_to_strategies = defaultdict(set)

    for book_title in top_books_info['Book-Title']:
        for strategy in content_strategy_names:
            try:
                recs = improved_content_filter(book_title, num_recommendations, strategy)
                for rec_book in recs['Book-Title']:
                    content_based_book_to_strategies[rec_book.strip().lower()].add((book_title, strategy))
            except Exception as e:
                print(f"Error for book '{book_title}' with strategy '{strategy}': {e}")


    collaborative_filtering_book_to_strategies = defaultdict(set)

    # SVD
    try:
        svd_recs_df = recommend_books_SVD(user_id, num_recommendations)
        for title in svd_recs_df['Book-Title']:
            cleaned = title.strip().lower()
            collaborative_filtering_book_to_strategies[cleaned].add("SVD")
    except Exception as e:
        print(f"Error generating SVD recommendations: {e}")

    # KNN
    try:
        knn_recs_df = recommend_books_KNN_custom(user_id, num_recommendations, k = 5)
        for title in knn_recs_df['Book-Title']:
            cleaned = title.strip().lower()
            collaborative_filtering_book_to_strategies[cleaned].add("KNN")
    except Exception as e:
        print(f"Error generating KNN recommendations: {e}")

    # Baseline
    try:
        base_recs_df = recommend_books_baseline(user_id, num_recommendations)
        for title in base_recs_df['Book-Title']:
            cleaned = title.strip().lower()
            collaborative_filtering_book_to_strategies[cleaned].add("BASELINE")
    except Exception as e:
        print(f"Error generating Baseline recommendations: {e}")

    # LightFM (based on ratings)
    try:
        lightfm_recs_df = recommend_books_hybrid_df(lightfm_ratings_model, lightfm_ratings_dataset, lightfm_ratings_item_features, user_id, n=num_recommendations)
        for title in lightfm_recs_df['Book-Title']:
            cleaned = title.strip().lower()
            collaborative_filtering_book_to_strategies[cleaned].add("LIGHTFM-RATINGS")
    except Exception as e:
        print(f"Error generating LightFM (ratings) recommendations: {e}")

    # LightFM (based on access history)
    try:
        lightfm_access_recs_df = recommend_books_hybrid_df(lightfm_history_model, lightfm_history_dataset, lightfm_history_item_features, user_id, n=num_recommendations)
        for title in lightfm_access_recs_df['Book-Title']:
            cleaned = title.strip().lower()
            collaborative_filtering_book_to_strategies[cleaned].add("LIGHTFM-HISTORY")
    except Exception as e:
        print(f"Error generating LightFM (history) recommendations: {e}")

    # Similarity
    try:
        #similar_recs_df = get_similar_users(user_id, n=num_recommendations)
        similar_recs_df = recommend_books_user_based(user_id, n=num_recommendations)

        for title in similar_recs_df['Book-Title']:
            cleaned = title.strip().lower()
            collaborative_filtering_book_to_strategies[cleaned].add("SIMILARITY")
    except Exception as e:
        print(f"Error generating Similarity recommendations: {e}")

    # Cold-Start
    # ADD COLD START for the novelty component?? recommended_books_cold_start = recommend_books_cold_start(new_user_age, new_user_location, n=5, age_threshold=5) - would need to add the users_info to this??
    # recommend_books_cold_start(new_age, new_location, n=5, age_threshold=5)
    try:
        new_user_age, new_user_location = get_user_demographics(user_id)
        cold_start_recs_df = recommend_books_cold_start(new_user_age, new_user_location, n=num_recommendations, age_threshold=5)
        for title in cold_start_recs_df['Book-Title']:
            cleaned = title.strip().lower()
            collaborative_filtering_book_to_strategies[cleaned].add("AGE-LOCATION")
    except Exception as e:
        print(f"Error generating Cold-Start recommendations: {e}")

    total_combinations = len(top_books_info) * len(content_strategy_names)

    # finds the maximum number of collaborative filtering strategies that recommend any single book
    max_collaborative_overlap = max((len(methods) for methods in collaborative_filtering_book_to_strategies.values()), default=1)

    binary_data = []

    all_books_set = set(content_based_book_to_strategies.keys()) | set(collaborative_filtering_book_to_strategies.keys())
    unique_books = list(all_books_set)

    for book in unique_books:
        row = {"Book-Title": book.title()}
        count_content = 0
        for base_book in top_books_info['Book-Title']:
            for strategy in content_strategy_names:
                key = (base_book, strategy)
                is_rec = int(key in content_based_book_to_strategies[book])
                row[f"{base_book} | {strategy}"] = is_rec
                count_content += is_rec

        count_collaborative = 0
        for strategy in collaborative_strategy_names:
            is_rec = int(strategy in collaborative_filtering_book_to_strategies.get(book, []))
            row[strategy] = is_rec
            count_collaborative += is_rec

        row["# Content-Based Strategies"] = count_content
        row["% Content-Based Strategies"] = round((count_content / total_combinations) * 100, 2)

        row["# Collaborative Filtering Strategies"] = count_collaborative
        row["% Collaborative Filtering Strategies"] = round((count_collaborative / len(collaborative_strategy_names)) * 100, 2)

        total_count = count_content + count_collaborative
        total_possible = total_combinations + len(collaborative_strategy_names)

        row["# Total Filtering Strategies"] = total_count
        row["% Total Filtering Strategies"] = round((total_count / total_possible) * 100, 2)

        content_score = (count_content / total_combinations) * 0.425
        collaborative_score = (count_collaborative / max_collaborative_overlap) * 0.425 # normalized collaborative part, because overlap is more rare than for content. so if we see a book appear in 3/7 of the methods, that is very good. so we count this as 3/3 for the collaborative component. so for final score it would contribute 3/3*0.425 = 0.425
        hybrid_bonus = 0.15 if (count_content > 0 and count_collaborative > 0) else 0 # add 0.2 to the score if the book if found by content-based and collaborative methods
        row["Final Score"] = round(content_score + collaborative_score + hybrid_bonus, 4)

        binary_data.append(row)

    return pd.DataFrame(binary_data).sort_values("Final Score", ascending=False)



In [31]:
from IPython.display import display, HTML

user_comparison_df = compare_user_top_books_binary_matrix(user_id=1, num_top_books=3, num_recommendations=10)
display(HTML(user_comparison_df.to_html(notebook=True)))

,Book-Title,Just Here Trying to Save a Few Lives : Tales of Life and Death from the ER | title,Just Here Trying to Save a Few Lives : Tales of Life and Death from the ER | author,Just Here Trying to Save a Few Lives : Tales of Life and Death from the ER | subjects,Just Here Trying to Save a Few Lives : Tales of Life and Death from the ER | synopsis,Just Here Trying to Save a Few Lives : Tales of Life and Death from the ER | title_author,Just Here Trying to Save a Few Lives : Tales of Life and Death from the ER | title_subjects,Just Here Trying to Save a Few Lives : Tales of Life and Death from the ER | title_synopsis,Just Here Trying to Save a Few Lives : Tales of Life and Death from the ER | author_subjects,Just Here Trying to Save a Few Lives : Tales of Life and Death from the ER | author_synopsis,Just Here Trying to Save a Few Lives : Tales of Life and Death from the ER | subjects_synopsis,Just Here Trying to Save a Few Lives : Tales of Life and Death from the ER | title_author_subject,Just Here Trying to Save a Few Lives : Tales of Life and Death from the ER | title_author_synopsis,Just Here Trying to Save a Few Lives : Tales of Life and Death from the ER | title_subjects_synopsis,Just Here Trying to Save a Few Lives : Tales of Life and Death from the ER | author_subjects_synopsis,Just Here Trying to Save a Few Lives : Tales of Life and Death from the ER | title_author_subjects_synopsis,Into the Green | title,Into the Green | author,Into the Green | subjects,Into the Green | synopsis,Into the Green | title_author,Into the Green | title_subjects,Into the Green | title_synopsis,Into the Green | author_subjects,Into the Green | author_synopsis,Into the Green | subjects_synopsis,Into the Green | title_author_subject,Into the Green | title_author_synopsis,Into the Green | title_subjects_synopsis,Into the Green | author_subjects_synopsis,Into the Green | title_author_subjects_synopsis,Sideways Stories from Wayside School | title,Sideways Stories from Wayside School | author,Sideways Stories from Wayside School | subjects,Sideways Stories from Wayside School | synopsis,Sideways Stories from Wayside School | title_author,Sideways Stories from Wayside School | title_subjects,Sideways Stories from Wayside School | title_synopsis,Sideways Stories from Wayside School | author_subjects,Sideways Stories from Wayside School | author_synopsis,Sideways Stories from Wayside School | subjects_synopsis,Sideways Stories from Wayside School | title_author_subject,Sideways Stories from Wayside School | title_author_synopsis,Sideways Stories from Wayside School | title_subjects_synopsis,Sideways Stories from Wayside School | author_subjects_synopsis,Sideways Stories from Wayside School | title_author_subjects_synopsis,SVD,KNN,BASELINE,AGE-LOCATION,LIGHTFM-RATINGS,LIGHTFM-HISTORY,SIMILARITY,# Content-Based Strategies,% Content-Based Strategies,# Collaborative Filtering Strategies,% Collaborative Filtering Strategies,# Total Filtering Strategies,% Total Filtering Strategies,Final Score
62,Your Erroneous Zones,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0.00,3,42.86,3,5.77,0.4250
139,Amelia Bedelia (I Can Read),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,1,1,1,0,0,0,0,0,0,1,8,17.78,1,14.29,9,17.31,0.3672
56,Moonheart (Newford),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,8,17.78,1,14.29,9,17.31,0.3672
22,You'Re Not A Kid Anymore When...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0.00,2,28.57,2,3.85,0.2833
130,"Bless Me, Ultima",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0.00,2,28.57,2,3.85,0.2833
76,The Once And Future King,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0.00,2,28.57,2,3.85,0.2833
73,Twenty Min

In [36]:
# should look for these manually in the dataset too to see if they are there but named slightly differently

friend_ratings = {
    "Friend 1": {
        "Handmaid’s Tale": 8, # likes
        "Shutter Island": 9, # likes
        "Rum Punch": 7, # likes
        "L.A. Confidential": 8, # likes
        "Dracula": 10, # likes
        "Crime and Punishment": 7, # likes
        "Misery": 8, # likes
        "Catcher In The Rye": 9, # likes
        "The Godfather": 9, # likes
        "Frankenstein": 9, # likes
        "Shake Hands With The Devil": 6, # likes
        "The Great Gatsby": 6, # likes
        "Lord Of The Flies": 9, # likes
        "War Of The Worlds": 7, # likes
        "The Road": 10, # likes
        "The Giver": 4, # dislikes
        "The Tommyknockers": 3, # dislikes
        "Fifty Shades of Grey": 2, # dislikes
        "Fahrenheit 451": 4, # dislikes
        "The Da Vinci Code": 4, # dislikes
    },
    "Friend 2": {
        "A Little Life": 8, # likes
        "Dying to Be Me": 7, # likes
        "There Will Come a Time": 7.5, # likes
        "When We Wake": 8, # likes
        "While We Run": 6, # likes
        "One of Us Is Lying": 9, # likes
        "Silent Alarm": 6, # likes
        "Before She Was Found": 7, # likes
        "We Were Liars": 8, # likes
        "If He Had Been with Me": 10, # likes
        "The Chef": 4, # dislikes
        "Sister Dear": 6, # dislikes
        "Reminders of Him": 6, # dislikes
        "The Other Passenger": 3, # dislikes
        "Give Me Your Hand": 5, # dislikes
    },
    "Friend 3": {
        "Invisible Life of Addie LaRue": 10,
        "Normal People": 10,
        "Tomorrow and Tomorrow and Tomorrow": 9,
        "Seven Year Slip": 7,
        "A Court of Thorns and Roses": 7,
        "A Court of Mist and Fury": 8,
        "I Who Have Never Known Men": 8,
        "The Secret History": 8,
        "Yellowface": 5,
        "People We Meet on Vacation": 6,
        "Fourth Wing": 5,
        "Pachinko": 8,
        "Happy Place": 2,
        "Silent Patient": 7,
        "Ashes & the Star Cursed King": 4,
        "Serpent & the Wings of Night": 6,
        "Conversations with Friends": 7,
        "Babel": 6,
        "The Love Hypothesis": 3,
        "Harry Potter & the Philosopher’s Stone": 10,
        "Icebreaker": 2,
        "21 Lessons for the 21st Century": 8,
        "Sapiens": 9,
        "Where the Crawdads Sing": 8,
        "Pride & Prejudice": 9
    },
    "Friend 4": {
        "Nineteen Eighty-Four": 9,  # likes
        "The Kite Runner": 10,  # likes
        "The Fault in Our Stars": 7,  # likes
        "The Hunger Games": 7,  # likes
        "Animal Farm": 8,  # likes
        "Love in the Time of Cholera": 7,  # likes
        "The Book Thief": 6,  # likes
        "One Hundred Years of Solitude": 6,  # likes
        "The Giver": 5,  # dislikes
        "The Curious Incident of the Dog in the Night-Time": 3  # dislikes
    },
    "Friend 5": {
        "The Lion, the Witch and the Wardrobe": 9,  # likes (Chronicles of Narnia Complete Collection) COULD CHOOSE ANOTHER - SEE WHAT'S IN DATASET
        "The Magician’s Nephew": 9,  # likes (Chronicles of Narnia Complete Collection) COULD CHOOSE ANOTHER - SEE WHAT'S IN DATASET
        "The Essential Calvin and Hobbes": 10,  # likes (Calvin & Hobbes Complete Collection) COULD CHOOSE ANOTHER - SEE WHAT'S IN DATASET
        "The Indispensable Calvin and Hobbes": 10,  # likes (Calvin & Hobbes Complete Collection) COULD CHOOSE ANOTHER - SEE WHAT'S IN DATASET
        "Brave New World": 8,  # likes
        "Grapes of Wrath": 7,  # likes
        "Beyond the Deepwoods": 8,  # likes (any Edge Chronicles book) COULD CHOOSE ANOTHER - SEE WHAT'S IN DATASET
        "Stormchaser": 8,  # likes  (any Edge Chronicles book) COULD CHOOSE ANOTHER - SEE WHAT'S IN DATASET
        "Eating Dirt": 9,  # likes
        "Freakonomics": 8,  # likes
        "The Shining": 9, # likes
        "Holes": 9, # likes
        "12 Rules for Life": 3,  # dislikes
        "A Tale of Two Cities": 4,  # dislikes
        "Fifth Business": 3,  # dislikes
        "Ugly Love": 1  # dislikes
    },
    "Friend 6": {
      "The Master and Margarita": 10,  # likes
      "The Picture of Dorian Gray": 8,  # likes
      "Normal People": 9,  # likes
      "The Song of Achilles": 10,  # likes
      "Just Kids": 9,  # likes
      "M Train": 9,  # likes
      "The Haunting of Hill House": 8,  # likes
      "Norwegian Wood": 7,  # likes
      "Kafka on the Shore": 10,  # likes
      "The Transit of Venus": 10,  # likes
      "One Hundred Years of Solitude": 4,  # dislikes
      "The Unbearable Lightness of Being": 4,  # dislikes
      "The Year of Magical Thinking": 6,  # dislikes
      "Frankenstein": 5,  # dislikes
      "Carmilla": 6  # dislikes
    },
    "Friend 7": {
      "Harry Potter and the Philosopher's Stone": 6,  # likes
      "Harry Potter and the Chamber of Secrets": 6,  # likes
      "Harry Potter and the Prisoner of Azkaban": 7,  # likes
      "Harry Potter and the Goblet of Fire": 6,  # likes
      "Harry Potter and the Order of the Phoenix": 10,  # likes
      "Harry Potter and the Half-Blood Prince": 9,  # likes
      "Harry Potter and the Deathly Hallows": 10,  # likes
      "Crazy Rich Asians": 7,  # likes
      "Dance of Thieves": 10,  # likes
      "Vow of Thieves": 8,  # likes
      "Divergent": 6,  # likes
      "City of Sparks": 6,  # likes
      "City of Ember": 6,  # likes
      "Verity": 8,  # likes
      "Hidden Figures": 3,  # dislikes
      "All the Light We Cannot See": 4  # dislikes
    },
    "Friend 8": {
      "Nineteen Eighty-Four": 9.5,
      "The Lord of the Rings": 8.5,
      "The Five People You Meet in Heaven": 7.5,
      "Verity": 7.0,
      "Throne of Glass": 9.0,
      "The Hunger Games": 6.5,
      "The Empyrean": 8.0, # fourth wing?
      "Camp Zero": 5.5,
      "The Midnight Library": 6.0,
      "Life of Pi": 6.0
    }
}

In [58]:
from collections import defaultdict
import pandas as pd

def compare_friend_top_books_binary_matrix(friend_name, friend_ratings_dict, num_top_books=3, num_recommendations=10):
    strategy_names = [
        "title",
        "author",
        "subjects",
        "synopsis",
        "title_author",
        "title_subjects",
        "title_synopsis",
        "author_subjects",
        "author_synopsis",
        "subjects_synopsis",
        "title_author_subject",
        "title_author_synopsis",
        "title_subjects_synopsis",
        "author_subjects_synopsis",
        "title_author_subjects_synopsis"
    ]
    # filtering friend ratings to include only books rated >= 5 and present in dataset
    friend_books = pd.Series(friend_ratings_dict[friend_name])
    enriched_titles = set(enriched_smaller_df['Book-Title'])

    filtered_books = friend_books[(friend_books >= 5) & (friend_books.index.isin(enriched_titles))]

    if filtered_books.empty:
        print(f"No rated books >= 5 from {friend_name} were found in the dataset.")
        return pd.DataFrame()

    # get top N from the filtered set
    top_books = filtered_books.sort_values(ascending=False).head(num_top_books).index.tolist()

    print(f"Using these top books for {friend_name}: {top_books}")

    # run recommendations and track which strategies recommend each book
    book_to_strategies = defaultdict(set)

    # get book recommendations for each book with different strategies
    for book_title in top_books:
        for strategy in strategy_names:
            try:
                recs = improved_content_filter(book_title, num_recommendations, strategy)
                for rec_book in recs['Book-Title']:
                    book_to_strategies[rec_book.strip().lower()].add((book_title, strategy))
            except Exception as e:
                print(f"Error for book '{book_title}' with strategy '{strategy}': {e}")

    #knn()
    #svd()

    # build comparison binary matrix
    binary_data = []
    unique_books = list(book_to_strategies.keys())
    total_combinations = len(top_books) * len(strategy_names)

    for book in unique_books:
        row = {"Book-Title": book.title()}
        count = 0
        for base_book in top_books:
            for strategy in strategy_names:
                key = (base_book, strategy)
                is_rec = int(key in book_to_strategies[book])
                row[f"{base_book} | {strategy}"] = is_rec
                count += is_rec
        row["# Content-Based Strategies"] = count
        row["% Content-Based Strategies"] = round((count / total_combinations) * 100, 2)
        binary_data.append(row)

    return pd.DataFrame(binary_data).sort_values("# Content-Based Strategies", ascending=False)


In [59]:
from IPython.display import display, HTML

friend_df = compare_friend_top_books_binary_matrix("Friend 1", friend_ratings)

if not friend_df.empty:
    display(HTML(friend_df.to_html(notebook=True)))


Using these top books for Friend 1: ['Dracula', 'The Godfather', 'Frankenstein']


,Book-Title,Dracula | title,Dracula | author,Dracula | subjects,Dracula | synopsis,Dracula | title_author,Dracula | title_subjects,Dracula | title_synopsis,Dracula | author_subjects,Dracula | author_synopsis,Dracula | subjects_synopsis,Dracula | title_author_subject,Dracula | title_author_synopsis,Dracula | title_subjects_synopsis,Dracula | author_subjects_synopsis,Dracula | title_author_subjects_synopsis,The Godfather | title,The Godfather | author,The Godfather | subjects,The Godfather | synopsis,The Godfather | title_author,The Godfather | title_subjects,The Godfather | title_synopsis,The Godfather | author_subjects,The Godfather | author_synopsis,The Godfather | subjects_synopsis,The Godfather | title_author_subject,The Godfather | title_author_synopsis,The Godfather | title_subjects_synopsis,The Godfather | author_subjects_synopsis,The Godfather | title_author_subjects_synopsis,Frankenstein | title,Frankenstein | author,Frankenstein | subjects,Frankenstein | synopsis,Frankenstein | title_author,Frankenstein | title_subjects,Frankenstein | title_synopsis,Frankenstein | author_subjects,Frankenstein | author_synopsis,Frankenstein | subjects_synopsis,Frankenstein | title_author_subject,Frankenstein | title_author_synopsis,Frankenstein | title_subjects_synopsis,Frankenstein | author_subjects_synopsis,Frankenstein | title_author_subjects_synopsis,# Content-Based Strategies,% Content-Based Strategies
67,The Sicilian,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14,31.11
64,Omerta,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,26.67
65,Fool'S Die,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,1,1,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,22.22
66,The Fourth K,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,17.78
68,"Sicilian, The",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,17.78
115,The Telling,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,1,1,1,8,17.78
113,Dandelion Wine,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,1,1,1,8,17.78
112,Requiem For A Ruler Of Worlds,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,1,1,1,8,17.78
71,The Storyteller,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,17.78
89,The Fortunate Pilgrim,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,15.56


### Attempt to modify collaborative strategies for friends:
We were successful for SVD but other collaborative methods require further work.

In [53]:
from surprise import SVD, Dataset, Reader
import pandas as pd

# Load your items metadata
items_info_cleaned_2_df = pd.read_csv(
    'https://raw.githubusercontent.com/tamaraamicic/Honours-Project-Datasets/refs/heads/main/items_info_cleaned_2.dat',
    sep='\t', on_bad_lines='skip', index_col=False
)

def recommend_books_SVD_for_friends(friend_ratings_dict, friend_name, n=10):
    # Convert friend's ratings to a DataFrame
    friend_data = pd.DataFrame([
        {'user': -1, 'item': row['Book_ID'], 'rating': rating}
        for title, rating in friend_ratings_dict[friend_name].items()
        for _, row in items_info_cleaned_2_df[items_info_cleaned_2_df['Book-Title'].str.strip().str.lower() == title.strip().lower()].iterrows()
    ])

    if friend_data.empty:
        print(f"No books found in dataset for {friend_name}'s ratings.")
        return pd.DataFrame()

    print(friend_data)

    # Define reader and load data
    reader = Reader(rating_scale=(1, 10))
    data = Dataset.load_from_df(friend_data[['user', 'item', 'rating']], reader)
    trainset = data.build_full_trainset()

    # Train SVD on friend's data
    svd_model = SVD(n_factors=50, random_state=42)
    svd_model.fit(trainset)

    # Get books not rated by the friend
    rated_books = set(friend_data['item'])
    all_books = set(items_info_cleaned_2_df['Book_ID'])
    unseen_books = list(all_books - rated_books)

    # Predict ratings
    predictions = [(book_id, svd_model.predict(-1, book_id).est) for book_id in unseen_books]
    top_books = sorted(predictions, key=lambda x: x[1], reverse=True)[:n]

    # Format recommendations
    book_ids = [book_id for book_id, _ in top_books]
    predicted_scores = dict(top_books)

    recommended_books = items_info_cleaned_2_df[items_info_cleaned_2_df['Book_ID'].isin(book_ids)].copy()
    recommended_books.loc[:, 'Predicted Rating'] = recommended_books['Book_ID'].map(predicted_scores)

    return recommended_books[['Book-Title', 'Book-Author', 'Publisher', 'Predicted Rating']]

friend1_recs = recommend_books_SVD_for_friends(friend_ratings, "Friend 1", n=10)
display(friend1_recs)

    user   item  rating
0     -1   1508      10
1     -1  14595      10
2     -1   1481       8
3     -1   2747       8
4     -1   2684       9
5     -1   3272       9
6     -1  16972       9
7     -1   2038       4
8     -1   4003       4
9     -1   2615       3
10    -1   1168       4
11    -1    406       4
12    -1   1313       4


,Book-Title,Book-Author,Publisher,Predicted Rating
0,Decision in Normandy,Carlo D'Este,HarperPerennial,6.592072
1,The Mummies of Urumchi,E. J. W. Barber,W. W. Norton & Company,6.592072
2,What If?: The World's Foremost Military Histor...,Robert Cowley,Berkley Publishing Group,6.592072
3,Beloved (Plume Contemporary Fiction),Toni Morrison,Plume,6.592072
4,Our Dumb Century: The Onion Presents 100 Years...,The Onion,Three Rivers Press,6.592072
5,New Vegetarian: Bold and Beautiful Recipes for...,Celia Brooks Brown,Ryland Peters & Small Ltd,6.592072
6,If I'd Known Then What I Know Now: Why Not Lea...,J. R. Parrish,Cypress House,6.592072
7,Airframe,Michael Crichton,Ballantine Books,6.592072
8,Timeline,MICHAEL CRICHTON,Ballantine Books,6.592072
9,OUT OF THE SILENT PLANET,C.S. Lewis,Scribner,6.592072


In [54]:
import re

def normalize_title(title):
    title = title.lower().strip()
    title = re.sub(r'\([^)]*\)', '', title)  # remove text in parentheses
    title = re.sub(r'[^a-z0-9 ]+', '', title)  # remove punctuation
    title = re.sub(r'\s+', ' ', title)  # normalize whitespace
    return title.strip()

def recommend_books_SVD_for_friends(friend_ratings, friend_name, n=10):
    friend_books_ratings = friend_ratings[friend_name]

    # Normalize the friend's book titles
    normalized_friend_books = {normalize_title(title): rating for title, rating in friend_books_ratings.items()}

    # Normalize the dataset titles
    items_info_cleaned_2_df['Normalized-Title'] = items_info_cleaned_2_df['Book-Title'].apply(normalize_title)

    # Match friend's books to Book_IDs
    matched_book_ids = set()
    for norm_title in normalized_friend_books:
        matches = items_info_cleaned_2_df[items_info_cleaned_2_df['Normalized-Title'] == norm_title]
        if not matches.empty:
            matched_book_ids.update(matches['Book_ID'].tolist())

    print(matched_book_ids)

    if not matched_book_ids:
        print(f"No books matched for {friend_name}.")
        return pd.DataFrame()

    # Now find all books that were NOT rated by the friend (based on Book_IDs)
    all_books = set(items_info_cleaned_2_df['Book_ID'])
    unrated_books = list(all_books - matched_book_ids)

    # Predict ratings for unrated books
    dummy_user_id = -1  # using dummy user ID for friend
    predictions = [(book, svd_model.predict(dummy_user_id, book).est) for book in unrated_books]
    top_predictions = sorted(predictions, key=lambda x: x[1], reverse=True)[:n]

    # Extract the top book IDs and scores
    book_ids = [book_id for book_id, _ in top_predictions]
    predicted_scores = {book_id: pred for book_id, pred in top_predictions}

    # Map back to book titles
    recommended_books = items_info_cleaned_2_df[items_info_cleaned_2_df['Book_ID'].isin(book_ids)].copy()
    recommended_books['Predicted Rating'] = recommended_books['Book_ID'].map(predicted_scores)

    return recommended_books[['Book-Title', 'Book-Author', 'Publisher', 'Predicted Rating']].sort_values('Predicted Rating', ascending=False)

friend1_recs = recommend_books_SVD_for_friends(friend_ratings, "Friend 1", n=10)
display(friend1_recs)

{14595, 5901, 1038, 1168, 3858, 406, 15390, 7711, 1313, 14243, 4003, 2484, 2615, 11961, 11450, 2747, 3272, 1481, 16972, 5970, 1508, 484, 2038, 2684, 9598}


,Book-Title,Book-Author,Publisher,Predicted Rating
7438,The Broken Promise Land,Marcia Muller,Warner Books,9.186484
2564,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,Scholastic,9.153899
2057,"Tucket's Gold (Tuckets Adventures, Book 4)",Gary Paulsen,Yearling Books,9.140662
3151,To Love This Life: Quotations From Helen Keller,Helen Keller,Scholastic Paperbacks,9.131020
1170,"House Harkonnen (Dune: House Trilogy, Book 2)",Brian Herbert,Spectra Books,9.087599
432,All of Me: A Voluptuous Tale,Venise T. Berry,Dutton Books,9.081545
3189,A Common Life : The Wedding Story (The Mitford...,Jan Karon,Viking Books,9.079339
13,Seabiscuit: An American Legend,LAURA HILLENBRAND,Ballantine Books,9.067149
10307,Coming Back: A Psychiatrist Explores Past-Life...,Raymond A. Moody,Bantam,9.062957
2954,Sideways Stories from Wayside School,Louis Sachar,HarperCollins,9.022365


In [55]:
from collections import defaultdict
import pandas as pd

def compare_friend_top_books_binary_matrix(friend_name, friend_ratings_dict, num_top_books=3, num_recommendations=10):
    content_strategy_names = [
        "title",
        "author",
        "subjects",
        "synopsis",
        "title_author",
        "title_subjects",
        "title_synopsis",
        "author_subjects",
        "author_synopsis",
        "subjects_synopsis",
        "title_author_subject",
        "title_author_synopsis",
        "title_subjects_synopsis",
        "author_subjects_synopsis",
        "title_author_subjects_synopsis"
    ]

    # Here, Age-Location uses the cold-start implementation
    collaborative_strategy_names = ["SVD", "KNN", "BASELINE", "AGE-LOCATION", "LIGHTFM-RATINGS", "LIGHTFM-HISTORY", "SIMILARITY"]  # Add more here and add "SVD" back here

    friend_books = pd.Series(friend_ratings_dict[friend_name])
    enriched_titles = set(enriched_smaller_df['Book-Title'])

    filtered_books = friend_books[(friend_books >= 5) & (friend_books.index.isin(enriched_titles))]

    if filtered_books.empty:
        print(f"No rated books >= 5 from {friend_name} were found in the dataset.")
        return pd.DataFrame()

    top_books = filtered_books.sort_values(ascending=False).head(num_top_books).index.tolist()
    print(f"Using these top books for {friend_name}: {top_books}")

    content_based_book_to_strategies = defaultdict(set)

    for book_title in top_books:
        for strategy in content_strategy_names:
            try:
                recs = improved_content_filter(book_title, num_recommendations, strategy)
                for rec_book in recs['Book-Title']:
                    content_based_book_to_strategies[rec_book.strip().lower()].add((book_title, strategy))
            except Exception as e:
                print(f"Error for book '{book_title}' with strategy '{strategy}': {e}")

    collaborative_filtering_book_to_strategies = defaultdict(set)

    dummy_user_id = -1  # Use a dummy user to simulate collaborative filtering recommendations

    # SVD
    try:
        #svd_recs_df = recommend_books_SVD(dummy_user_id, num_recommendations)
        svd_recs_df = recommend_books_SVD_for_friends(friend_ratings, friend_name, n=num_recommendations)
        for title in svd_recs_df['Book-Title']:
            cleaned = title.strip().lower()
            collaborative_filtering_book_to_strategies[cleaned].add("SVD")
    except Exception as e:
        print(f"Error generating SVD recommendations: {e}")

    # KNN
    try:
        knn_recs_df = recommend_books_KNN_custom(dummy_user_id, num_recommendations, k=5)
        for title in knn_recs_df['Book-Title']:
            cleaned = title.strip().lower()
            collaborative_filtering_book_to_strategies[cleaned].add("KNN")
    except Exception as e:
        print(f"Error generating KNN recommendations: {e}")

    # Baseline
    try:
        base_recs_df = recommend_books_baseline(dummy_user_id, num_recommendations)
        for title in base_recs_df['Book-Title']:
            cleaned = title.strip().lower()
            collaborative_filtering_book_to_strategies[cleaned].add("BASELINE")
    except Exception as e:
        print(f"Error generating Baseline recommendations: {e}")

    # LightFM (based on ratings)
    try:
        lightfm_recs_df = recommend_books_hybrid_df(lightfm_ratings_model, lightfm_ratings_dataset, lightfm_ratings_item_features, dummy_user_id, n=num_recommendations)
        for title in lightfm_recs_df['Book-Title']:
            cleaned = title.strip().lower()
            collaborative_filtering_book_to_strategies[cleaned].add("LIGHTFM-RATINGS")
    except Exception as e:
        print(f"Error generating LightFM (ratings) recommendations: {e}")

    # LightFM (based on access history)
    try:
        lightfm_access_recs_df = recommend_books_hybrid_df(lightfm_history_model, lightfm_history_dataset, lightfm_history_item_features, dummy_user_id, n=num_recommendations)
        for title in lightfm_access_recs_df['Book-Title']:
            cleaned = title.strip().lower()
            collaborative_filtering_book_to_strategies[cleaned].add("LIGHTFM-HISTORY")
    except Exception as e:
        print(f"Error generating LightFM (history) recommendations: {e}")

    # Similarity
    try:
        similar_recs_df = recommend_books_user_based(dummy_user_id, n=num_recommendations)
        for title in similar_recs_df['Book-Title']:
            cleaned = title.strip().lower()
            collaborative_filtering_book_to_strategies[cleaned].add("SIMILARITY")
    except Exception as e:
        print(f"Error generating Similarity recommendations: {e}")

    # Cold-Start
    # ADD COLD START for the novelty component?? recommended_books_cold_start = recommend_books_cold_start(new_user_age, new_user_location, n=5, age_threshold=5) - would need to add the users_info to this??
    # recommend_books_cold_start(new_age, new_location, n=5, age_threshold=5)
    try:
        new_user_age, new_user_location = get_user_demographics(dummy_user_id)
        cold_start_recs_df = recommend_books_cold_start(new_user_age, new_user_location, n=num_recommendations, age_threshold=5)
        for title in cold_start_recs_df['Book-Title']:
            cleaned = title.strip().lower()
            collaborative_filtering_book_to_strategies[cleaned].add("AGE-LOCATION")
    except Exception as e:
        print(f"Error generating Cold-Start recommendations: {e}")

    total_combinations = len(top_books) * len(content_strategy_names)

    # finds the maximum number of collaborative filtering strategies that recommend any single book
    max_collaborative_overlap = max((len(methods) for methods in collaborative_filtering_book_to_strategies.values()), default=1)

    binary_data = []

    all_books_set = set(content_based_book_to_strategies.keys()) | set(collaborative_filtering_book_to_strategies.keys())
    unique_books = list(all_books_set)

    for book in unique_books:
        row = {"Book-Title": book.title()}
        count_content = 0
        for base_book in top_books:
            for strategy in content_strategy_names:
                key = (base_book, strategy)
                is_rec = int(key in content_based_book_to_strategies[book])
                row[f"{base_book} | {strategy}"] = is_rec
                count_content += is_rec

        row["# Content-Based Strategies"] = count_content
        row["% Content-Based Strategies"] = round((count_content / total_combinations) * 100, 2)

        count_collaborative = 0
        for strategy in collaborative_strategy_names:
            is_rec = int(strategy in collaborative_filtering_book_to_strategies.get(book, []))
            row[strategy] = is_rec
            count_collaborative += is_rec

        row["# Collaborative Filtering Strategies"] = count_collaborative
        row["% Collaborative Filtering Strategies"] = round((count_collaborative / len(collaborative_strategy_names)) * 100, 2)

        total_count = count_content + count_collaborative
        total_possible = total_combinations + len(collaborative_strategy_names)

        row["# Total Filtering Strategies"] = total_count
        row["% Total Filtering Strategies"] = round((total_count / total_possible) * 100, 2)

        content_score = (count_content / total_combinations) * 0.5
        collaborative_score = (count_collaborative / max_collaborative_overlap) * 0.5 # normalized collaborative part, because overlap is more rare than for content. so if we see a book appear in 3/7 of the methods, that is very good. so we count this as 3/3 for the collaborative component. so for final score it would contribute 3/3*0.5 = 0.5
        row["Final Score"] = round(content_score + collaborative_score, 4)

        binary_data.append(row)

    return pd.DataFrame(binary_data).sort_values("Final Score", ascending=False)

In [56]:
from IPython.display import display, HTML

friend_df = compare_friend_top_books_binary_matrix("Friend 1", friend_ratings)

if not friend_df.empty:
    display(HTML(friend_df.to_html(notebook=True)))

Using these top books for Friend 1: ['Dracula', 'The Godfather', 'Frankenstein']
{14595, 5901, 1038, 1168, 3858, 406, 15390, 7711, 1313, 14243, 4003, 2484, 2615, 11961, 11450, 2747, 3272, 1481, 16972, 5970, 1508, 484, 2038, 2684, 9598}
Error generating KNN recommendations: -1
Error generating LightFM (ratings) recommendations: User or item ids cannot be negative. Check your inputs for negative numbers or very large numbers that can overflow.
Error generating LightFM (history) recommendations: User or item ids cannot be negative. Check your inputs for negative numbers or very large numbers that can overflow.
Error generating Similarity recommendations: -1
User -1 not found.
Error generating Cold-Start recommendations: 'NoneType' object has no attribute 'lower'


,Book-Title,Dracula | title,Dracula | author,Dracula | subjects,Dracula | synopsis,Dracula | title_author,Dracula | title_subjects,Dracula | title_synopsis,Dracula | author_subjects,Dracula | author_synopsis,Dracula | subjects_synopsis,Dracula | title_author_subject,Dracula | title_author_synopsis,Dracula | title_subjects_synopsis,Dracula | author_subjects_synopsis,Dracula | title_author_subjects_synopsis,The Godfather | title,The Godfather | author,The Godfather | subjects,The Godfather | synopsis,The Godfather | title_author,The Godfather | title_subjects,The Godfather | title_synopsis,The Godfather | author_subjects,The Godfather | author_synopsis,The Godfather | subjects_synopsis,The Godfather | title_author_subject,The Godfather | title_author_synopsis,The Godfather | title_subjects_synopsis,The Godfather | author_subjects_synopsis,The Godfather | title_author_subjects_synopsis,Frankenstein | title,Frankenstein | author,Frankenstein | subjects,Frankenstein | synopsis,Frankenstein | title_author,Frankenstein | title_subjects,Frankenstein | title_synopsis,Frankenstein | author_subjects,Frankenstein | author_synopsis,Frankenstein | subjects_synopsis,Frankenstein | title_author_subject,Frankenstein | title_author_synopsis,Frankenstein | title_subjects_synopsis,Frankenstein | author_subjects_synopsis,Frankenstein | title_author_subjects_synopsis,# Content-Based Strategies,% Content-Based Strategies,SVD,KNN,BASELINE,AGE-LOCATION,LIGHTFM-RATINGS,LIGHTFM-HISTORY,SIMILARITY,# Collaborative Filtering Strategies,% Collaborative Filtering Strategies,# Total Filtering Strategies,% Total Filtering Strategies,Final Score
145,Harry Potter And The Chamber Of Secrets (Book 2),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,1,0,0,0,0,0,0,1,14.29,1,1.92,0.5000
2,Seabiscuit: An American Legend,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,1,0,0,0,0,0,0,1,14.29,1,1.92,0.5000
5,"Tucket'S Gold (Tuckets Adventures, Book 4)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,1,0,0,0,0,0,0,1,14.29,1,1.92,0.5000
108,Coming Back: A Psychiatrist Explores Past-Life...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,1,0,0,0,0,0,0,1,14.29,1,1.92,0.5000
107,Sideways Stories From Wayside School,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,1,0,0,0,0,0,0,1,14.29,1,1.92,0.5000
104,A Common Life : The Wedding Story (The Mitford...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,1,0,0,0,0,0,0,1,14.29,1,1.92,0.5000
160,To Love This Life: Quotations From Helen Keller,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,1,0,0,0,0,0,0,1,14.29,1,1.92,0.5000
47,"House Harkonnen (Dune: House Trilogy, Book 2)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,1,0,0,0,0,0,0,1,14.29,1,1.92,0.5000
149,The Broken Promise Land,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,1,0,0,0,0,0,0,1,14.29,1,1.92,0.5000
144,All Of Me: A Voluptuous Tale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,1,0,0,0,0,0,0,1,14.29,1,1.92,0.5000


Future work: Taking user feed back. Keeping in mind that people won't pick up and finish books that they didn't think they would like.

Rating validation vs rating availability.

Can we trust these ratings as much as amazon product purshases?

Don't like/like the theme/idea but it was poorly done.

The nuances are important to consider, even with clean data.